In [1]:
import krippendorff
import numpy as np
import pandas as pd
import re
import pickle
import random
import json
from collections import Counter
from sklearn.metrics import f1_score
from itertools import combinations
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score


In [2]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

In [3]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             
classes_3 = ['Unrelated', 'Consistent', 'Inconsistent']

### Recalculate recall

In [47]:
recall_per_model_type = dict()

In [7]:
recall_per_model_type = dict()
for model_type in prediction_ground_truth_tuples:
    recall_per_model_type[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples[model_type]:
        try:
            recall_per_model_type[model_type][ground_truth]['ground_truth_count'] +=1 
            if ground_truth == prediction:
                recall_per_model_type[model_type][prediction]['retrieved'] +=1 
        except KeyError:
            recall_per_model_type[model_type][ground_truth] = dict() 
            recall_per_model_type[model_type][ground_truth]['ground_truth_count'] = 1
            recall_per_model_type[model_type][ground_truth]['retrieved'] = 0
            if ground_truth == prediction:
                recall_per_model_type[model_type][prediction]['retrieved'] +=1 

In [8]:
for model_type in recall_per_model_type:
    print(model_type)
    for class_name in recall_per_model_type[model_type]:
        print(class_name)
        recall = recall_per_model_type[model_type][class_name]['retrieved']/recall_per_model_type[model_type][class_name]['ground_truth_count']
        print(recall)
        recall_per_model_type[model_type][class_name]['recall'] = recall
        

humans 5 classes
Indirect inconsistency
0.42718446601941745
Surface contradiction
0.5429553264604811
Unrelated
0.5884244372990354
Factual inconsistency
0.39012738853503187
Consistent
0.6968421052631579
ChatGPT-4 5 classes
Indirect inconsistency
0.6947141316073355
Surface contradiction
0.33791523482245134
Unrelated
0.6527331189710611
Factual inconsistency
0.15605095541401273
Consistent
0.848421052631579
ChatGPT-3.5 5 classes
Indirect inconsistency
0.20604099244875945
Surface contradiction
0.5063001145475372
Unrelated
0.08038585209003216
Factual inconsistency
0.6273885350318471
Consistent
0.30526315789473685
LLaMA 8B 5 classes
Indirect inconsistency
0.15102481121898598
Surface contradiction
0.21076746849942726
Unrelated
0.4228295819935691
Factual inconsistency
0.4888535031847134
Consistent
0.6884210526315789
LLaMA 70B 5 classes
Indirect inconsistency
0.7723840345199569
Surface contradiction
0.4146620847651776
Unrelated
0.37942122186495175
Factual inconsistency
0.1767515923566879
Consiste

In [14]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\recall_per_model_type.pkl', 'wb') as f:
    pickle.dump(recall_per_model_type, f)

In [5]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\recall_per_model_type.pkl', 'rb') as f:
    recall_per_model_type = pickle.load(f)

In [10]:
model_type = 'bootstrap'
recall_per_model_type['bootstrap'] = dict()
            
for ground_truth, prediction in bootstrap_predictions_tuples:
    try:
        recall_per_model_type[model_type][ground_truth]['ground_truth_count'] +=1 
        if ground_truth == prediction:
            recall_per_model_type[model_type][prediction]['retrieved'] +=1 
    except KeyError:
        recall_per_model_type[model_type][ground_truth] = dict() 
        recall_per_model_type[model_type][ground_truth]['ground_truth_count'] = 1
        recall_per_model_type[model_type][ground_truth]['retrieved'] = 0
        if ground_truth == prediction:
            recall_per_model_type[model_type][prediction]['retrieved'] +=1 

In [12]:
for class_name in recall_per_model_type['bootstrap']:
    print(class_name)
    recall = recall_per_model_type[model_type][class_name]['retrieved']/recall_per_model_type[model_type][class_name]['ground_truth_count']
    print(recall)
    recall_per_model_type[model_type][class_name]['recall'] = recall

Indirect inconsistency
0.6911733046286329
Surface contradiction
0.7451428571428571
Unrelated
0.7653721682847896
Factual inconsistency
0.6610410094637224
Consistent
0.8311300639658848


In [ ]:
recall_per_model_type

In [39]:
f1_score_per_model_type = dict()

In [40]:
for model_type in prediction_ground_truth_tuples:
    f1_score_per_model_type[model_type] = dict()

    print(model_type)
    # prediction_ground_truth_tuples_f1_scores[model_type] = dict()
    true_labels = []
    predictions = []
    for ground_truth, prediction in prediction_ground_truth_tuples[model_type]:
        true_labels.append(ground_truth)
        predictions.append(prediction)
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_f1_score = f1_score(true_labels, predictions, labels=classes_5, average=None)
    print("Per class", res_f1_score)
    my_dict = dict(zip(classes_5, res_f1_score))
    f1_score_per_model_type[model_type] = my_dict
    print('Just mean', np.mean(res_f1_score))
    res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_5, average='macro')
    print("Macro", res_f1_score_macro)
    f1_score_per_model_type[model_type]['macro'] = res_f1_score_macro

    res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_5, average='micro')
    print("Micro", res_f1_score_micro)
    f1_score_per_model_type[model_type]['micro'] = res_f1_score_micro


humans 5 classes
Per class [0.58842444 0.68036999 0.43349754 0.37837838 0.55406195]
Just mean 0.5269464588841088
Macro 0.5269464588841088
Micro 0.5140425531914894
ChatGPT-4 5 classes
Per class [0.6429137  0.77574591 0.47580347 0.24716267 0.47275641]
Just mean 0.522876432640254
Macro 0.522876432640254
Micro 0.5236879432624113
ChatGPT-3.5 5 classes
Per class [0.14880952 0.44072948 0.27053824 0.35801908 0.41955387]
Just mean 0.327530040202492
Macro 0.327530040202492
Micro 0.3466666666666667
LLaMA 8B 5 classes
Per class [0.3879056  0.50894942 0.20588235 0.34906197 0.28527132]
Just mean 0.34741413177190594
Macro 0.34741413177190594
Micro 0.34638297872340423
LLaMA 70B 5 classes
Per class [0.50212766 0.71262136 0.50762141 0.26025791 0.51493599]
Just mean 0.4995128663025096
Macro 0.4995128663025096
Micro 0.5083687943262412


In [45]:
model_type = 'bootstrap'
f1_score_per_model_type['bootstrap'] = dict()
true_labels = []
predictions = []
for ground_truth, prediction in bootstrap_predictions_tuples:
    true_labels.append(ground_truth)
    predictions.append(prediction)
res_f1_score = f1_score(true_labels, predictions, labels=classes_5, average=None)
print("Per class", res_f1_score)
my_dict = dict(zip(classes_5, res_f1_score))
f1_score_per_model_type[model_type] = my_dict
print('Just mean', np.mean(res_f1_score))
res_f1_score_macro = f1_score(true_labels, predictions, labels=classes_5, average='macro')
print("Macro", res_f1_score_macro)
f1_score_per_model_type[model_type]['macro'] = res_f1_score_macro

res_f1_score_micro = f1_score(true_labels, predictions, labels=classes_5, average='micro')
print("Micro", res_f1_score_micro)
f1_score_per_model_type[model_type]['micro'] = res_f1_score_micro
   

Per class [0.76555798 0.81462905 0.69751779 0.6507764  0.7545423 ]
Just mean 0.7366047033160935
Macro 0.7366047033160935
Micro 0.7307801418439717


In [46]:
# with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\f1_score_per_model_type.pkl', 'wb') as f:
#     pickle.dump(f1_score_per_model_type, f)

### Recalculate precision

In [6]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\prediction_ground_truth_tuples.json", 'r') as f:
    prediction_ground_truth_tuples = json.load(f)

In [5]:
precision_per_model_type = dict()
for model_type in prediction_ground_truth_tuples:
    precision_per_model_type[model_type] = dict()
    for ground_truth, prediction in prediction_ground_truth_tuples[model_type]:
        try:
            precision_per_model_type[model_type][prediction]['predicted'] +=1 
            if ground_truth == prediction:
                precision_per_model_type[model_type][prediction]['actual'] +=1 
        except KeyError:
            precision_per_model_type[model_type][prediction] = dict() 
            precision_per_model_type[model_type][prediction]['predicted'] = 1
            precision_per_model_type[model_type][prediction]['actual'] = 0
            if ground_truth == prediction:
                precision_per_model_type[model_type][prediction]['actual'] +=1 

In [19]:
for model_type in precision_per_model_type:
    print(model_type)
    for class_name in precision_per_model_type[model_type]:
        print(class_name)
        precision = precision_per_model_type[model_type][class_name]['actual']/precision_per_model_type[model_type][class_name]['predicted']
        print(precision)
        precision_per_model_type[model_type][class_name]['precision'] = precision
        

humans 5 classes
Indirect inconsistency
0.44
Factual inconsistency
0.36731634182908546
Consistent
0.6646586345381527
Surface contradiction
0.5656324582338902
Unrelated
0.5884244372990354
ChatGPT-4 5 classes
Indirect inconsistency
0.36179775280898874
Factual inconsistency
0.593939393939394
Surface contradiction
0.7866666666666666
Unrelated
0.6333853354134166
Consistent
0.7145390070921985
ChatGPT-3.5 5 classes
Factual inconsistency
0.25047679593134137
Surface contradiction
0.3581847649918963
Indirect inconsistency
0.3938144329896907
Unrelated
1.0
Consistent
0.7923497267759563
LLaMA 8B 5 classes
Indirect inconsistency
0.3233256351039261
Factual inconsistency
0.27144120247568526
Unrelated
0.3583106267029973
Surface contradiction
0.4412470023980815
Consistent
0.40370370370370373
LLaMA 70B 5 classes
Indirect inconsistency
0.3780359028511088
Unrelated
0.7421383647798742
Factual inconsistency
0.49333333333333335
Surface contradiction
0.6791744840525328
Consistent
0.6612612612612613
bootstrap
I

In [26]:
# calculate total
for model_type in precision_per_model_type:
    total_actual = 0
    total_predicted = 0
    for class_name in precision_per_model_type[model_type]:
        total_actual += precision_per_model_type[model_type][class_name]['actual']
        total_predicted += precision_per_model_type[model_type][class_name]['predicted']
    total_precision = total_actual/total_predicted
    precision_per_model_type[model_type]['total'] = total_precision    

In [28]:
# with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\precision_per_model_type.pkl", 'wb') as f:
#     pickle.dump(precision_per_model_type, f)

In [4]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\bootstrap_predictions_tuples.pkl", 'rb') as f:
    bootstrap_predictions_tuples = pickle.load(f)

In [18]:
precision_per_model_type['bootstrap'] = dict()
for ground_truth, prediction in bootstrap_predictions_tuples:
    try:
        precision_per_model_type['bootstrap'][prediction]['predicted'] +=1 
        if ground_truth == prediction:
            precision_per_model_type['bootstrap'][prediction]['actual'] +=1 
    except KeyError:
        precision_per_model_type['bootstrap'][prediction] = dict() 
        precision_per_model_type['bootstrap'][prediction]['predicted'] = 1
        precision_per_model_type['bootstrap'][prediction]['actual'] = 0
        if ground_truth == prediction:
            precision_per_model_type['bootstrap'][prediction]['actual'] +=1 

### Recalculating agreement - but now we concatenate all tuples for a class

In [6]:
df_gold['tuples_majority_4'] = None

In [42]:
prediction_ground_truth_tuples = dict()
prediction_ground_truth_tuples['humans 5 classes'] = []
prediction_ground_truth_tuples['ChatGPT-4 5 classes'] = []
prediction_ground_truth_tuples['ChatGPT-3.5 5 classes'] = []
prediction_ground_truth_tuples['LLaMA 8B 5 classes'] = []
prediction_ground_truth_tuples['LLaMA 70B 5 classes'] = []

In [43]:
for i in range(len(df_gold)):
    # we have 5 tuples per sample 
    # ground truth - from majority, the 5th one is prediction 
    strings = eval(df_gold['All answers'][i])
    # create N-1 tuples
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]
    # go through each element
    # value is held out "prediction"
    for value, version in versions:
        # print('dropping', value)
        list_annotator_dropped = version
        # print("list_annotator_dropped", list_annotator_dropped)
        counter = Counter(list_annotator_dropped)
        # identify the majority vote and sample randomly if more than 1
        max_count = max(counter.values())
        top_labels = [label for label, count in counter.items() if count == max_count]
        majority = random.choice(top_labels)
 
        prediction_ground_truth_tuples['humans 5 classes'].append((majority, value)) #ground truth, prediction
        prediction_ground_truth_tuples['ChatGPT-4 5 classes'].append((majority, df_gold['ChatGPT-4 5 classes'][i]))
        prediction_ground_truth_tuples['ChatGPT-3.5 5 classes'].append((majority, df_gold['ChatGPT-3.5 5 classes'][i]))
        prediction_ground_truth_tuples['LLaMA 8B 5 classes'].append((majority, df_gold['LLaMA 5 classes'][i]))
        prediction_ground_truth_tuples['LLaMA 70B 5 classes'].append((majority, df_gold['LLaMA 70B 5 classes'][i]))
        
        
    

In [45]:
aggregated_prediction_ground_truth_tuples = dict()
aggregated_prediction_ground_truth_tuples['humans 5 classes'] = dict()
aggregated_prediction_ground_truth_tuples['ChatGPT-4 5 classes'] = dict()
aggregated_prediction_ground_truth_tuples['ChatGPT-3.5 5 classes'] = dict()
aggregated_prediction_ground_truth_tuples['LLaMA 8B 5 classes'] = dict()
aggregated_prediction_ground_truth_tuples['LLaMA 70B 5 classes'] = dict()

In [46]:
for model_type in aggregated_prediction_ground_truth_tuples:
    for ground_truth, prediction in prediction_ground_truth_tuples[model_type]:
        try:
            aggregated_prediction_ground_truth_tuples[model_type][ground_truth].append(prediction)
        except:
            aggregated_prediction_ground_truth_tuples[model_type][ground_truth] = [prediction]
        

In [96]:
aggregated_prediction_ground_truth_tuples_precisions = dict()
aggregated_prediction_ground_truth_tuples_recalls = dict()


for model_type in aggregated_prediction_ground_truth_tuples:
    print(model_type)
    aggregated_prediction_ground_truth_tuples_precisions[model_type] = dict()
    aggregated_prediction_ground_truth_tuples_recalls[model_type] = dict()
    
    for ground_truth in aggregated_prediction_ground_truth_tuples[model_type]:
        print(ground_truth)
        predictions = aggregated_prediction_ground_truth_tuples[model_type][ground_truth]
        true_labels = [ground_truth] * len(predictions) 
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
        res_precision = precision_score(true_labels, predictions, labels=[ground_truth], average=None)
        print('Precision', res_precision)
        aggregated_prediction_ground_truth_tuples_precisions[model_type][ground_truth] = res_precision
        
        res_recall = recall_score(true_labels, predictions, labels=[ground_truth], average=None)
        print('Recall', res_recall)
        aggregated_prediction_ground_truth_tuples_recalls[model_type][ground_truth] = res_recall

humans 5 classes
Indirect inconsistency
Precision [1.]
Recall [0.42718447]
Surface contradiction
Precision [1.]
Recall [0.54295533]
Unrelated
Precision [1.]
Recall [0.58842444]
Factual inconsistency
Precision [1.]
Recall [0.39012739]
Consistent
Precision [1.]
Recall [0.69684211]
ChatGPT-4 5 classes
Indirect inconsistency
Precision [1.]
Recall [0.69471413]
Surface contradiction
Precision [1.]
Recall [0.33791523]
Unrelated
Precision [1.]
Recall [0.65273312]
Factual inconsistency
Precision [1.]
Recall [0.15605096]
Consistent
Precision [1.]
Recall [0.84842105]
ChatGPT-3.5 5 classes
Indirect inconsistency
Precision [1.]
Recall [0.20604099]
Surface contradiction
Precision [1.]
Recall [0.50630011]
Unrelated
Precision [1.]
Recall [0.08038585]
Factual inconsistency
Precision [1.]
Recall [0.62738854]
Consistent
Precision [1.]
Recall [0.30526316]
LLaMA 8B 5 classes
Indirect inconsistency
Precision [1.]
Recall [0.15102481]
Surface contradiction
Precision [1.]
Recall [0.21076747]
Unrelated
Precisio

In [97]:
for model_type in aggregated_prediction_ground_truth_tuples:
    print(model_type)
    total_predictions = []
    total_ground_truth = []
    
    for ground_truth in aggregated_prediction_ground_truth_tuples[model_type]:
        predictions = aggregated_prediction_ground_truth_tuples[model_type][ground_truth]
        true_labels = [ground_truth] * len(predictions) 
        total_predictions.extend(predictions)
        total_ground_truth.extend(true_labels)
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_precision_total = precision_score(total_ground_truth, total_predictions, average='macro')
    print('Precision', res_precision_total)
    res_recall_total = recall_score(total_ground_truth, total_predictions, average='macro')
    print('Recall', res_recall_total)
    aggregated_prediction_ground_truth_tuples_recalls[model_type]['total'] = res_recall_total 
    aggregated_prediction_ground_truth_tuples_precisions[model_type]['total'] = res_precision_total

humans 5 classes
Precision 0.5252063743800328
Recall 0.5291067447154247
ChatGPT-4 5 classes
Precision 0.6180656311841328
Recall 0.5379668986892879
ChatGPT-3.5 5 classes
Precision 0.558965144137777
Recall 0.34507573040258255
LLaMA 8B 5 classes
Precision 0.35960563407687884
Recall 0.3923792835056549
LLaMA 70B 5 classes
Precision 0.5907886692556221
Recall 0.5031701024908285


In [98]:
aggregated_prediction_ground_truth_tuples_recalls

{'humans 5 classes': {'Indirect inconsistency': array([0.42718447]),
  'Surface contradiction': array([0.54295533]),
  'Unrelated': array([0.58842444]),
  'Factual inconsistency': array([0.39012739]),
  'Consistent': array([0.69684211]),
  'total': 0.5291067447154247},
 'ChatGPT-4 5 classes': {'Indirect inconsistency': array([0.69471413]),
  'Surface contradiction': array([0.33791523]),
  'Unrelated': array([0.65273312]),
  'Factual inconsistency': array([0.15605096]),
  'Consistent': array([0.84842105]),
  'total': 0.5379668986892879},
 'ChatGPT-3.5 5 classes': {'Indirect inconsistency': array([0.20604099]),
  'Surface contradiction': array([0.50630011]),
  'Unrelated': array([0.08038585]),
  'Factual inconsistency': array([0.62738854]),
  'Consistent': array([0.30526316]),
  'total': 0.34507573040258255},
 'LLaMA 8B 5 classes': {'Indirect inconsistency': array([0.15102481]),
  'Surface contradiction': array([0.21076747]),
  'Unrelated': array([0.42282958]),
  'Factual inconsistency':

In [47]:
aggregated_prediction_ground_truth_tuples_f1_scores = dict()

In [48]:
for model_type in aggregated_prediction_ground_truth_tuples:
    print(model_type)
    aggregated_prediction_ground_truth_tuples_f1_scores[model_type] = dict()
    for ground_truth in aggregated_prediction_ground_truth_tuples[model_type]:
        print(ground_truth)
        predictions = aggregated_prediction_ground_truth_tuples[model_type][ground_truth]
        true_labels = [ground_truth] * len(predictions) 
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
        res_f1_score = f1_score(true_labels, predictions, average=None)
        print(res_f1_score)
        aggregated_prediction_ground_truth_tuples_f1_scores[model_type][ground_truth] = res_f1_score

humans 5 classes
Indirect inconsistency
[0.         0.         0.59863946 0.         0.        ]
Surface contradiction
[0.         0.         0.         0.70378619 0.        ]
Unrelated
[0.         0.         0.         0.         0.74089069]
Factual inconsistency
[0.         0.56128293 0.         0.         0.        ]
Consistent
[0.82133995 0.         0.         0.         0.        ]
ChatGPT-4 5 classes
Indirect inconsistency
[0.         0.         0.81985996 0.         0.        ]
Surface contradiction
[0.         0.         0.         0.50513699 0.        ]
Unrelated
[0.         0.         0.         0.78988327]
Factual inconsistency
[0.         0.26997245 0.         0.         0.        ]
Consistent
[0.91799544 0.         0.         0.        ]
ChatGPT-3.5 5 classes
Indirect inconsistency
[0.         0.         0.34168157 0.        ]
Surface contradiction
[0.         0.         0.         0.67224335]
Unrelated
[0.         0.         0.         0.         0.14880952]
Factual incon

Now total F1 macro score

In [49]:
for model_type in aggregated_prediction_ground_truth_tuples:
    print(model_type)
    total_predictions = []
    total_ground_truth = []
    
    for ground_truth in aggregated_prediction_ground_truth_tuples[model_type]:
        predictions = aggregated_prediction_ground_truth_tuples[model_type][ground_truth]
        true_labels = [ground_truth] * len(predictions) 
        total_predictions.extend(predictions)
        total_ground_truth.extend(true_labels)
        # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_f1_score_total = f1_score(total_ground_truth, total_predictions, average='macro')
    print(res_f1_score_total)
    aggregated_prediction_ground_truth_tuples_f1_scores[model_type]['total'] = res_f1_score_total

humans 5 classes
0.5269464588841088
ChatGPT-4 5 classes
0.522876432640254
ChatGPT-3.5 5 classes
0.327530040202492
LLaMA 8B 5 classes
0.34741413177190605
LLaMA 70B 5 classes
0.4995128663025096


### Synthetic upper-bound

bootstrapping within the set of 4 -> what would happen if we were to create another set of ground truth? 

we take today’s ground truth as gold and tomorrow’s as prediction, what would be the f-score?


Question: previously we used majority as a ground truth and held out answer as a prediction. Now we are changing it. Is it legit to compare both?

bootstrap sampling for simulations
if we only ever had 4 annotations -> we do bootstrap samples -> 10 samples with replacement from the initial 4. bootstrap size is also 4!
initial 4 = ground truth.
10 predictions for 1 ground truth. 
this we do 5 times. 
50 pairs per sample. 
Group by ground truth. 


each value has 66% chance to be in the bootstrap sample

In [54]:
len(version)

4

In [12]:
bootstrap_predictions_tuples = []

for i in range(len(df_gold)):
    # we have 5 tuples per sample 
    strings = eval(df_gold['All answers'][i])
    # create N-1 tuples
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]
    # print(len(versions))
    # go through each element, value is held out "ground truth"
    for value, version in versions:
        list_annotator_dropped = version
        counter = Counter(list_annotator_dropped)
        # identify the majority vote and sample randomly if more than 1
        max_count = max(counter.values())
        top_labels = [label for label, count in counter.items() if count == max_count]
        # this is today's ground truth
        majority = random.choice(top_labels)
        # 10 bootstraps of the same pool where we took the majority from 
        bootstraps = [random.choices(version, k=len(version)) for _ in range(10)]
        bootstrap_majorities = []
        for bootstrap in bootstraps:
            counter = Counter(bootstrap)
            max_count = max(counter.values())
            top_labels = [label for label, count in counter.items() if count == max_count]
            # this is tomorrow's ground truth
            tomorrow_majority = random.choice(top_labels)
            bootstrap_majorities.append(tomorrow_majority)
        # print(len(bootstrap_majorities))
        to_append = []
        for tomorrow_majority in bootstrap_majorities:
            to_append.append((majority, tomorrow_majority))
        bootstrap_predictions_tuples.extend(to_append)        
            


      

In [13]:
bootstrap_aggregated = dict()

In [76]:
boostrap_ground_truth = []
boostrap_predictions = []

for ground_truth, prediction in bootstrap_predictions_tuples:
    boostrap_ground_truth.append(ground_truth)
    boostrap_predictions.append(prediction)
    
res_f1_score_total = f1_score(boostrap_ground_truth, boostrap_predictions, average='macro')
print(res_f1_score_total)
bootstrap_f1_scores['total'] = res_f1_score_total

0.7389506575982552


In [80]:
for class_name in classes_5:
    bootstrap_aggregated[class_name] = []

In [81]:
for ground_truth, prediction in bootstrap_predictions_tuples:
    try:
        bootstrap_aggregated[ground_truth].append(prediction)
    except:
        bootstrap_aggregated[ground_truth] = [prediction]

In [75]:
bootstrap_f1_scores = dict()

In [82]:
for ground_truth in bootstrap_aggregated:
    print(ground_truth)
    predictions = bootstrap_aggregated[ground_truth]
    true_labels = [ground_truth] * len(predictions) 
    # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_f1_score = f1_score(true_labels, predictions, average=None)
    print(res_f1_score)
    bootstrap_f1_scores[ground_truth] = res_f1_score

Unrelated
[0.         0.         0.         0.         0.88220928]
Consistent
[0.90816563 0.         0.         0.         0.        ]
Indirect inconsistency
[0.       0.       0.811517 0.       0.      ]
Factual inconsistency
[0.        0.7991449 0.        0.        0.       ]
Surface contradiction
[0.         0.         0.         0.85283514 0.        ]


In [99]:
bootstrap_recalls = dict()
for ground_truth in bootstrap_aggregated:
    print(ground_truth)
    predictions = bootstrap_aggregated[ground_truth]
    true_labels = [ground_truth] * len(predictions) 
    # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_recall = recall_score(true_labels, predictions, average=None)
    print(res_recall)
    bootstrap_recalls[ground_truth] = res_recall

Unrelated
[0.        0.        0.        0.        0.7892437]
Consistent
[0.83177966 0.         0.         0.         0.        ]
Indirect inconsistency
[0.        0.        0.6828175 0.        0.       ]
Factual inconsistency
[0.         0.66547988 0.         0.         0.        ]
Surface contradiction
[0.         0.         0.         0.74342857 0.        ]


C:\Users\Nursulu_1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nursulu_1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nursulu_1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Reca

In [101]:
boostrap_ground_truth = []
boostrap_predictions = []

for ground_truth, prediction in bootstrap_predictions_tuples:
    boostrap_ground_truth.append(ground_truth)
    boostrap_predictions.append(prediction)
    
res_recall_total = recall_score(boostrap_ground_truth, boostrap_predictions, average='macro')
print(res_recall_total)
# bootstrap_f1_scores['total'] = res_recall_total

0.7425498617507185


In [ ]:
boostrap_ground_truth = []
boostrap_predictions = []

for ground_truth, prediction in bootstrap_predictions_tuples:
    boostrap_ground_truth.append(ground_truth)
    boostrap_predictions.append(prediction)
    
res_recall_total = recall_score(boostrap_ground_truth, boostrap_predictions, average='macro')
print(res_recall_total)
# bootstrap_f1_scores['total'] = res_recall_total

In [100]:
bootstrap_precisions = dict()
for ground_truth in bootstrap_aggregated:
    print(ground_truth)
    predictions = bootstrap_aggregated[ground_truth]
    true_labels = [ground_truth] * len(predictions) 
    # we use average = Macro when there are several classses. sums f1 scores for each class/divide by N classes
    res_precision = precision_score(true_labels, predictions, average=None)
    print(res_precision)
    bootstrap_precisions[ground_truth] = res_precision

Unrelated
[0. 0. 0. 0. 1.]
Consistent
[1. 0. 0. 0. 0.]
Indirect inconsistency
[0. 0. 1. 0. 0.]
Factual inconsistency
[0. 1. 0. 0. 0.]
Surface contradiction
[0. 0. 0. 1. 0.]


In [14]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\bootstrap_predictions_tuples.pkl', 'wb') as f:
    pickle.dump(bootstrap_predictions_tuples, f)

In [84]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\bootstrap_aggregated.pkl', 'wb') as f:
    pickle.dump(bootstrap_aggregated, f)

In [83]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\bootstrap_f1_scores.pkl', 'wb') as f:
    pickle.dump(bootstrap_f1_scores, f)

In [50]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\aggregated_prediction_ground_truth_tuples_f1_scores.pkl', 'wb') as f:
    pickle.dump(aggregated_prediction_ground_truth_tuples_f1_scores, f)

In [51]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\aggregated_prediction_ground_truth_tuples.json', 'w') as f:
    json.dump(aggregated_prediction_ground_truth_tuples, f)

In [52]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\prediction_ground_truth_tuples.json', 'w') as f:
    json.dump(prediction_ground_truth_tuples, f)

In [14]:
prediction_ground_truth_tuples['humans 5 classes'][:10]

[('Indirect inconsistency', 'Indirect inconsistency'),
 ('Indirect inconsistency', 'Indirect inconsistency'),
 ('Indirect inconsistency', 'Indirect inconsistency'),
 ('Indirect inconsistency', 'Indirect inconsistency'),
 ('Indirect inconsistency', 'Factual inconsistency'),
 ('Surface contradiction', 'Consistent'),
 ('Surface contradiction', 'Surface contradiction'),
 ('Surface contradiction', 'Surface contradiction'),
 ('Surface contradiction', 'Factual inconsistency'),
 ('Surface contradiction', 'Surface contradiction')]

In [3]:
df_gold

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,Model_agreement_with_human,model_agreement_with_human_3_classes,ChatGPT-3.5 5 classes,...,llama_70b_crowd_wisdom_agreement,chatgpt4_individuals_agreement,chatgpt3_5_individuals_agreement,llama_8b_individuals_agreement,llama_70b_individuals_agreement,Annotator_explanations,Explanations,Who_explained,NLI_ChatGPT4_turbo,Avg_synthetic_agreement
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,1.000000,Factual inconsistency,...,1.0,0.800000,0.2,0.800000,0.800000,0,['Limiting employee influence goes against ide...,['v10'],Contradiction,1.0
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,0.800000,Surface contradiction,...,0.0,0.000000,0.6,0.200000,0.000000,0,['Broad support for all agriculture in incompa...,['v4'],Contradiction,0.7
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,1.000000,Factual inconsistency,...,0.0,0.200000,0.0,0.000000,0.200000,0,0,0,Contradiction,1.0
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,Indirect inconsistency,Unrelated,0.200000,0.600000,Indirect inconsistency,...,0.0,0.200000,0.2,0.600000,0.200000,0,0,0,Unrelated,0.7
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,Indirect inconsistency,Inconsistent,0.833333,0.833333,Factual inconsistency,...,1.0,0.833333,0.0,0.166667,0.833333,0,['Text A promotes cycling. Text B encourages c...,['v8'],Unrelated,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Inconsistent,0.400000,1.000000,Surface contradiction,...,1.0,0.400000,0.0,0.000000,0.600000,0,"[""Increasing interest rates may cause people t...",['v7'],Unrelated,0.7
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,1.000000,Surface contradiction,...,0.0,0.000000,0.6,0.400000,0.000000,0,0,0,Contradiction,0.7
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,0.800000,Surface contradiction,...,0.0,0.800000,0.0,0.800000,0.200000,0,0,0,Unrelated,1.0
696,Text 1: The level of social basic security mus...,Factual inconsistency,40.000,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,0.600000,Surface contradiction,...,0.0,0.200000,0.0,0.000000,0.200000,0,0,0,Contradiction,0.3


In [29]:
df_gold['Avg_synthetic_agreement'] = 0

In [32]:
for i in tqdm(range(len(df_gold))):
    gold_label = df_gold['Answer'][i]
    answers = eval(df_gold['All answers'][i])
    triples = list(combinations(answers, 3))
    majorities = [Counter(triples[j]).most_common()[0][0] for j in range(len(triples))]
    counter = 0
    for el in majorities:
        if el == gold_label:
            counter += 1
    avg_agreement = counter/len(majorities)
    df_gold['Avg_synthetic_agreement'][i] = avg_agreement

  0%|          | 0/698 [00:00<?, ?it/s]C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_151784\629694028.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Avg_synthetic_agreement'][i] = avg_agreement
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_151784\629694028.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Avg_synthetic_agreement'][i] = avg_agreement
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_151784\629694028.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7' has dtype incompatible with int64, please explic

In [38]:
avg_synthetic_agreement_per_class = dict()

In [3]:
df_gold['Avg_synthetic_agreement_F1_score'] = 0

In [6]:
for i in tqdm(range(len(df_gold))):
    gold_label = df_gold['Answer'][i]
    answers = eval(df_gold['All answers'][i])
    triples = list(combinations(answers, 3))
    # select the most common element
    majorities = [Counter(triples[j]).most_common()[0][0] for j in range(len(triples))]
    true_labels = [gold_label for i in range(len(triples))]
    avg_f1 = f1_score(true_labels, majorities, average='macro')
    df_gold['Avg_synthetic_agreement_F1_score'][i] = avg_f1

  0%|          | 0/698 [00:00<?, ?it/s]C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_156856\3615047600.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Avg_synthetic_agreement_F1_score'][i] = avg_f1
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_156856\3615047600.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gold['Avg_synthetic_agreement_F1_score'][i] = avg_f1
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_156856\3615047600.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.4117647058823529' has dtype incompatible with i

In [8]:
avg_synthetic_agreement_per_class_f1 = dict()

In [10]:
for class_name in classes_5:
    df_class = df_gold[df_gold['Answer'] == class_name]
    avg_synthetic_agreement_per_class_f1[class_name] = np.mean(df_class['Avg_synthetic_agreement_F1_score'])

In [11]:
avg_synthetic_agreement_per_class_f1['total'] = np.mean(df_gold['Avg_synthetic_agreement_F1_score'])

In [12]:
avg_synthetic_agreement_per_class_f1

{'Unrelated': 0.6958469107810558,
 'Consistent': 0.7616567312653807,
 'Indirect inconsistency': 0.5433668147646646,
 'Factual inconsistency': 0.5024280242616284,
 'Surface contradiction': 0.6261453783606141,
 'total': 0.6137141328315575}

In [13]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\avg_synthetic_agreement_per_class_f1.json', 'w') as f:
    json.dump(avg_synthetic_agreement_per_class_f1, f)

In [43]:
for class_name in classes_5:
    df_class = df_gold[df_gold['Answer'] == class_name]
    avg_synthetic_agreement_per_class[class_name] = np.mean(df_class['Avg_synthetic_agreement'])

In [46]:
avg_synthetic_agreement_per_class['total'] = np.mean(df_gold['Avg_synthetic_agreement'])

In [47]:
avg_synthetic_agreement_per_class

{'Unrelated': 0.826271186440678,
 'Consistent': 0.8625000000000002,
 'Indirect inconsistency': 0.7480446927374302,
 'Factual inconsistency': 0.717622950819672,
 'Surface contradiction': 0.7924408014571951,
 'total': 0.7833333333333333}

In [48]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\avg_synthetic_agreement_per_class.json", 'w') as f:
    json.dump(avg_synthetic_agreement_per_class, f)

In [33]:
Counter(df_gold['Avg_synthetic_agreement'])

Counter({1.0: 278,
         0.7: 205,
         0.5: 62,
         0.3: 48,
         0.9: 36,
         0.8: 35,
         0.6: 23,
         0.4: 7,
         0.75: 1,
         0.85: 1,
         0.45: 1,
         0.8166666666666667: 1})

In [34]:
Counter(df_gold['Confidence'])

Counter({60.0: 257,
         40.0: 168,
         80.0: 142,
         100.0: 109,
         50.0: 10,
         83.333: 6,
         66.667: 2,
         33.333: 2,
         20.0: 1,
         70.0: 1})

In [15]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\style_imitator\eng_statement_to_translation.json", 'r') as f:
    eng_surface = json.load(f)

In [16]:
eng_surface

{'A rent control is necessary to protect low-income individuals.': "🏠🔒 **Rent Control is Essential for Community Stability!** 🔒🏠\n\nIn these challenging economic times, hardworking Americans are feeling the pinch of skyrocketing living costs more than ever. It is our duty to ensure affordable housing remains a cornerstone of our promise to America’s low-income families.\n\n👉 House Republicans understand the importance of stability in housing as part of a broader strategy to stimulate job growth, control inflation, and boost economic independence. While we strive for fiscal responsibility, we also recognize that strategic policies like rent control are necessary to protect the most vulnerable in our society from the volatile swings of the market.\n\n🇺🇸 By preserving affordable housing through controlled measures, we ensure that our community members can live securely, contributing to a robust, vibrant economy. It's not just about saving dollars—it's about making sense for America's futu

In [41]:
df_gold = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv")

In [3]:
df_gold

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,Model_agreement_with_human,model_agreement_with_human_3_classes,ChatGPT-3.5 5 classes,...,llama_8b_crowd_wisdom_agreement,llama_70b_crowd_wisdom_agreement,chatgpt4_individuals_agreement,chatgpt3_5_individuals_agreement,llama_8b_individuals_agreement,llama_70b_individuals_agreement,Annotator_explanations,Explanations,Who_explained,NLI_ChatGPT4_turbo
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,1.000000,Factual inconsistency,...,1.0,1.0,0.800000,0.2,0.800000,0.800000,0,['Limiting employee influence goes against ide...,['v10'],Contradiction
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,0.800000,Surface contradiction,...,0.0,0.0,0.000000,0.6,0.200000,0.000000,0,['Broad support for all agriculture in incompa...,['v4'],Contradiction
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,1.000000,Factual inconsistency,...,0.0,0.0,0.200000,0.0,0.000000,0.200000,0,0,0,Contradiction
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,Indirect inconsistency,Unrelated,0.200000,0.600000,Indirect inconsistency,...,1.0,0.0,0.200000,0.2,0.600000,0.200000,0,0,0,Unrelated
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,Indirect inconsistency,Inconsistent,0.833333,0.833333,Factual inconsistency,...,0.0,1.0,0.833333,0.0,0.166667,0.833333,0,['Text A promotes cycling. Text B encourages c...,['v8'],Unrelated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Inconsistent,0.400000,1.000000,Surface contradiction,...,0.0,1.0,0.400000,0.0,0.000000,0.600000,0,"[""Increasing interest rates may cause people t...",['v7'],Unrelated
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,1.000000,Surface contradiction,...,0.6,0.0,0.000000,0.6,0.400000,0.000000,0,0,0,Contradiction
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,0.800000,Surface contradiction,...,1.0,0.0,0.800000,0.0,0.800000,0.200000,0,0,0,Unrelated
696,Text 1: The level of social basic security mus...,Factual inconsistency,40.000,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,0.600000,Surface contradiction,...,0.0,0.0,0.200000,0.0,0.000000,0.200000,0,0,0,Contradiction


In [ ]:
df_gold_agreement = df_gold['Answer', 'Confidence', 'human_human_agreement', ]

### Evaluate BART NLI

In [5]:
predictions_bart = []
true_labels = []

for i in range(len(df_gold)):
    true_label = df_gold['Answer_3_classes'].iloc[i]
    true_labels.append(true_label)
        
    pred = df_gold['bart_NLI'].iloc[i]
    predictions_bart.append(pred)


In [11]:
f1_bart = f1_score(true_labels, predictions_bart, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
f1_bart_total = f1_score(true_labels, predictions_bart, average='macro')

In [ ]:
true_labels = list(df_gold['Answer_3_classes'])
predictions_chatgpt = list(df_gold['NLI_ChatGPT4_turbo'])
predictions_chatgpt = [nli_to_labels[el] for el in predictions_chatgpt]

f1_chatgpt = f1_score(true_labels, predictions_chatgpt, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
f1_chatgpt_total = f1_score(true_labels, predictions_chatgpt, average='macro')


### Evaluate ChatGPT4 turbo NLI

In [21]:
for i in range(len(df_gold)):
    if df_gold['NLI_ChatGPT4_turbo'][i] == str(0): 
        print(i)

In [6]:
nli_to_labels = {'Contradiction': 'Inconsistent', 'Unrelated': 'Unrelated', 'Entailment': 'Consistent'}

In [23]:
predictions = []
true_labels = []

for i in range(len(df_gold)):
    true_label = df_gold['Answer_3_classes'].iloc[i]
    true_labels.append(true_label)
        
    pred = df_gold['NLI_ChatGPT4_turbo'].iloc[i]
    try:
        pred = nli_to_labels[pred]
    except KeyError:
        print(pred)

    predictions.append(pred)


In [24]:
f1_score = f1_score(true_labels, predictions, average='macro')

In [25]:
f1_score

0.3930103885260836

In [26]:
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)

In [27]:
accuracy

0.44269340974212035

In [5]:
predictions_vs_true_nli_chatgpt = []
predictions_vs_true_nli_deberta = []

In [20]:
true_labels = list(df_gold['Answer_3_classes'])
predictions_chatgpt = list(df_gold['NLI_ChatGPT4_turbo'])
predictions_chatgpt = [nli_to_labels[el] for el in predictions_chatgpt]

f1_chatgpt = f1_score(true_labels, predictions_chatgpt, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
f1_chatgpt_total = f1_score(true_labels, predictions_chatgpt, average='macro')

predictions_deberta = list(df_gold['deberta-v3-large-nli_3_classes'])
# predictions_deberta = [[el] for el in predictions_deberta]

f1_deberta = f1_score(true_labels, predictions_deberta, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
f1_deberta_total = f1_score(true_labels, predictions_deberta, average='macro')
    

In [30]:
recall_chatgpt = recall_score(true_labels, predictions_chatgpt, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
recall_chatgpt_total = recall_score(true_labels, predictions_chatgpt, average='macro')

recall_deberta = recall_score(true_labels, predictions_deberta, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
recall_deberta_total = recall_score(true_labels, predictions_deberta, average='macro')

In [38]:
precision_chatgpt = precision_score(true_labels, predictions_chatgpt, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
precision_chatgpt_total = precision_score(true_labels, predictions_chatgpt, average='macro')

precision_deberta = precision_score(true_labels, predictions_deberta, labels=['Unrelated', 'Consistent', 'Inconsistent'], average=None)
precision_deberta_total = precision_score(true_labels, predictions_deberta, average='macro')

In [31]:
NLI_experiments = dict()

In [32]:
NLI_experiments['ChatGPT-4 turbo'] = dict()

label_to_f1 = dict(zip(list(f1_chatgpt), ['Unrelated', 'Consistent', 'Inconsistent']))
label_to_f1['total'] = f1_chatgpt_total
NLI_experiments['ChatGPT-4 turbo']['f1_score'] = label_to_f1

NLI_experiments['Deberta'] = dict()
label_to_f1 = dict(zip(list(f1_deberta), ['Unrelated', 'Consistent', 'Inconsistent']))
label_to_f1['total'] = f1_deberta_total
NLI_experiments['Deberta']['f1_score'] = label_to_f1

In [36]:
label_to_recall = dict(zip(list(recall_chatgpt), ['Unrelated', 'Consistent', 'Inconsistent']))
label_to_recall['total'] = recall_chatgpt_total
NLI_experiments['ChatGPT-4 turbo']['recall'] = label_to_recall

label_to_recall = dict(zip(list(recall_deberta), ['Unrelated', 'Consistent', 'Inconsistent']))
label_to_recall['total'] = recall_deberta_total
NLI_experiments['Deberta']['recall'] = label_to_recall

In [39]:
label_to_precision = dict(zip(list(precision_chatgpt), ['Unrelated', 'Consistent', 'Inconsistent']))
label_to_precision['total'] = precision_chatgpt_total
NLI_experiments['ChatGPT-4 turbo']['precision'] = label_to_precision

label_to_precision = dict(zip(list(precision_deberta), ['Unrelated', 'Consistent', 'Inconsistent']))
label_to_precision['total'] = precision_deberta_total
NLI_experiments['Deberta']['precision'] = label_to_precision

In [40]:
NLI_experiments

{'ChatGPT-4 turbo': {'f1_score': {0.37662337662337664: 'Unrelated',
   0.2702702702702703: 'Consistent',
   0.5321375186846039: 'Inconsistent',
   'total': 0.3930103885260836},
  'recall': {0.9830508474576272: 'Unrelated',
   0.15625: 'Consistent',
   0.3677685950413223: 'Inconsistent',
   'total': 0.5023564808329831},
  'precision': {0.23293172690763053: 'Unrelated',
   1.0: 'Consistent',
   0.9621621621621622: 'Inconsistent',
   'total': 0.7316979630232643}},
 'Deberta': {'f1_score': {0.01680672268907563: 'Unrelated',
   0.5904761904761905: 'Consistent',
   0.8416119962511714: 'Inconsistent',
   'total': 0.48296496980547915},
  'recall': {0.00847457627118644: 'Unrelated',
   0.6458333333333334: 'Consistent',
   0.9276859504132231: 'Inconsistent',
   'total': 0.527331286672581},
  'precision': {1.0: 'Unrelated',
   0.543859649122807: 'Consistent',
   0.7701543739279588: 'Inconsistent',
   'total': 0.7713380076835886}}}

In [41]:
# with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\NLI_experiments.pkl', 'wb') as f:
#     pickle.dump(NLI_experiments, f)

In [34]:
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['NLI_ChatGPT4_turbo'])
    predictions = [nli_to_labels[el] for el in predictions]
    # accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    # print(f"Accuracy: {accuracy:.4f}")
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"F1_score: {f1:.4f}")
    

Unrelated
Accuracy: 0.9831
F1_score: 0.4957
Consistent
Accuracy: 0.1562
F1_score: 0.0901
Inconsistent
Accuracy: 0.3678
F1_score: 0.2689


In [42]:
wrong_predictions_deberta = []
wrong_predictions_chatgpt = []


for i in range(len(df_gold)):
    true_label = df_gold['Answer_3_classes'].iloc[i]        
    pred_chatgpt = df_gold['NLI_ChatGPT4_turbo'].iloc[i]
    pred_chatgpt = nli_to_labels[pred_chatgpt]
    if pred_chatgpt != true_label:
        # ground_truth, predicted
        wrong_predictions_chatgpt.append((df_gold['Answer'].iloc[i], pred_chatgpt))
        
    pred_deberta = df_gold['deberta-v3-large-nli_3_classes'].iloc[i]
    if pred_deberta != true_label:
        # ground_truth, predicted
        wrong_predictions_deberta.append((df_gold['Answer'].iloc[i], pred_deberta))
    


In [4]:
df_gold.columns


Index(['Input', 'Answer', 'Confidence', 'All answers', 'Answer_3_classes',
       'ChatGPT-4 5 classes', 'ChatGPT-4 3 classes',
       'Model_agreement_with_human', 'model_agreement_with_human_3_classes',
       'ChatGPT-3.5 5 classes', 'ChatGPT-3.5 3 classes', 'LLaMA 5 classes',
       'LLaMA 3 classes', 'LLaMA 70B 5 classes', 'LLaMA 70B 3 classes',
       'deberta-v3-large-nli_3_classes',
       'Model_agreement_with_human_gpt3.5_turbo',
       'model_agreement_with_human_3_classes_gpt3.5_turbo',
       'Model_agreement_with_human_LLaMA_8B',
       'Model_agreement_with_human_3_classes_LLaMA_8B',
       'Model_agreement_with_human_LLaMA_70B',
       'Model_agreement_with_human_3_classes_LLaMA_70B',
       'Model_agreement_with_human_3_classes_deberta_nli',
       'human_human_agreement', 'human_crowd_wisdom_agreement',
       'chatgpt4_crowd_wisdom_agreement', 'chatgpt3_5_crowd_wisdom_agreement',
       'llama_8b_crowd_wisdom_agreement', 'llama_70b_crowd_wisdom_agreement',
       'ch

In [7]:
ground_truth_vs_prediction_NLI_all = dict()

predictions_deberta = []
predictions_chatgpt = []
predictions_bart = []


for i in range(len(df_gold)):
    true_label = df_gold['Answer_3_classes'].iloc[i]        
    pred_chatgpt = df_gold['NLI_ChatGPT4_turbo'].iloc[i]
    pred_chatgpt = nli_to_labels[pred_chatgpt]
    predictions_chatgpt.append((df_gold['Answer'].iloc[i], pred_chatgpt))
        
    pred_deberta = df_gold['deberta-v3-large-nli_3_classes'].iloc[i]
    predictions_deberta.append((df_gold['Answer'].iloc[i], pred_deberta))
    
    pred_bart = df_gold['bart_NLI'].iloc[i]
    predictions_bart.append((df_gold['Answer'].iloc[i], pred_bart))
    
ground_truth_vs_prediction_NLI_all['chatgpt'] = predictions_chatgpt
ground_truth_vs_prediction_NLI_all['deberta'] = predictions_deberta
ground_truth_vs_prediction_NLI_all['bart'] = predictions_bart



    

In [8]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\ground_truth_vs_prediction_NLI_all.pkl', 'wb') as f:
    pickle.dump(ground_truth_vs_prediction_NLI_all, f)

In [45]:
ground_truth_vs_prediction_NLI_wrong = dict()
ground_truth_vs_prediction_NLI_wrong['chatgpt'] = wrong_predictions_chatgpt
ground_truth_vs_prediction_NLI_wrong['deberta'] = wrong_predictions_deberta

In [46]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\ground_truth_vs_prediction_NLI_wrong.json', 'w') as f:
    json.dump(ground_truth_vs_prediction_NLI_wrong, f)

In [49]:
wrong_predictions_chatgpt

[('Indirect inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Surface contradiction', 'Unrelated'),
 ('Factual inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Consistent', 'Unrelated'),
 ('Consistent', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Consistent', 'Unrelated'),
 ('Surface contradiction', 'Unrelated'),
 ('Surface contradiction', 'Unrelated'),
 ('Surface contradiction', 'Unrelated'),
 ('Consistent', 'Inconsistent'),
 ('Consistent', 'Unrelated'),
 ('Factual inconsistency', 'Unrelated'),
 ('Factual inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Factual inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Indirect inconsistency', 'Unrelated'),
 ('Surface cont

In [43]:
Counter(wrong_predictions_chatgpt)

Counter({('Indirect inconsistency', 'Unrelated'): 156,
         ('Surface contradiction', 'Unrelated'): 76,
         ('Consistent', 'Unrelated'): 76,
         ('Factual inconsistency', 'Unrelated'): 74,
         ('Consistent', 'Inconsistent'): 5,
         ('Unrelated', 'Inconsistent'): 2})

In [44]:
Counter(wrong_predictions_deberta)


Counter({('Unrelated', 'Inconsistent'): 100,
         ('Consistent', 'Inconsistent'): 34,
         ('Unrelated', 'Consistent'): 17,
         ('Indirect inconsistency', 'Consistent'): 16,
         ('Surface contradiction', 'Consistent'): 12,
         ('Factual inconsistency', 'Consistent'): 7})

In [36]:
Counter(wrong_predictions)

Counter({'Indirect inconsistency': 156,
         'Consistent': 81,
         'Surface contradiction': 76,
         'Factual inconsistency': 74,
         'Unrelated': 2})

In [37]:
Counter(df_gold['Answer'])

Counter({'Surface contradiction': 183,
         'Indirect inconsistency': 179,
         'Factual inconsistency': 122,
         'Unrelated': 118,
         'Consistent': 96})

In [39]:
wrong_predictions_deberta = []

for i in range(len(df_gold)):
    true_label = df_gold['Answer_3_classes'].iloc[i]        
    pred = df_gold['deberta-v3-large-nli_3_classes'].iloc[i]
    if pred != true_label:
        wrong_predictions_deberta.append(df_gold['Answer'].iloc[i] )


In [40]:
Counter(wrong_predictions_deberta)

Counter({'Unrelated': 117,
         'Consistent': 34,
         'Indirect inconsistency': 16,
         'Surface contradiction': 12,
         'Factual inconsistency': 7})

In [41]:
Counter(df_gold['deberta-v3-large-nli_3_classes'])

Counter({'Inconsistent': 583, 'Consistent': 114, 'Unrelated': 1})

In [42]:
Counter(df_gold['Answer_3_classes'])

Counter({'Inconsistent': 484, 'Unrelated': 118, 'Consistent': 96})

### Evaluate BART NLI 3 types

### Evaluate DeBERTa NLI 3 types

In [70]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_deberta-v3-large-nli_3_types.pkl", 'rb') as f:
    answers_deberta_3_types = pickle.load(f)

In [71]:
len(answers_deberta_3_types)

700

In [74]:
all_samples_df = pd.read_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\deberta-v3-large-nli-3-types.csv")

In [79]:
answers_deberta_3_types = dict(zip(list(all_samples_df['Input']), list(all_samples_df['deberta-v3-large-nli'])))

In [83]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in answers_deberta_3_types.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_deberta_3_types:
        answers_deberta_3_types[new_key] = answers_deberta_3_types.pop(old_key)

In [86]:
df_gold['deberta-v3-large-nli_3_classes'] = 0

In [87]:
predictions = []
true_labels = []

for el in answers_deberta_3_types:
    true_label = df_gold[df_gold['Input'] == el]['Answer_3_classes'].iloc[0]
    true_labels.append(true_label)
        
    pred = answers_deberta_3_types[el]

    pred = pred.lower().capitalize()
    predictions.append(pred)
    df_gold.loc[df_gold['Input'] == el, 'deberta-v3-large-nli_3_classes'] = pred


C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_146048\3783832118.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'deberta-v3-large-nli_3_classes'] = pred


In [89]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

### Evaluate LLaMa 70B

#### 5 classes

In [6]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_3_70B_instruct_5_types.pkl", 'rb') as f:
    answers_llama3_3_70B_instruct_5_types = pickle.load(f)

In [7]:
### replace "\n" with "\n\n"
key_changes = dict()

for el in answers_llama3_3_70B_instruct_5_types.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_llama3_3_70B_instruct_5_types:
        answers_llama3_3_70B_instruct_5_types[new_key] = answers_llama3_3_70B_instruct_5_types.pop(old_key)

In [8]:
len(answers_llama3_3_70B_instruct_5_types)

698

In [9]:
df_gold['LLaMA 70B 5 classes'] = 0

In [10]:
to_edit = []
classes_5_casefold = classes_5 + ["Surface Contradiction", "Factual Inconsistency", "Indirect Inconsistency"]
for el in answers_llama3_3_70B_instruct_5_types:
    res = answers_llama3_3_70B_instruct_5_types[el].content
    found_classes = [cls for cls in classes_5_casefold if cls in res]
    if not found_classes:
        print(el)
    elif len(found_classes)>1:
        print(el)
        # el = el.replace('\n\n', '\n')
        # to_edit.append(el)


In [11]:
predictions = []
true_labels = []
classes_5_casefold = classes_5 + ["Surface Contradiction", "Factual Inconsistency", "Indirect Inconsistency"]

for el in answers_llama3_3_70B_instruct_5_types:
    try:
        
        true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
        true_labels.append(true_label)
        
        res = answers_llama3_3_70B_instruct_5_types[el].content
        found_classes = [cls for cls in classes_5_casefold if cls in res]
        if len(found_classes) > 1:
            print(found_classes)
        pred = found_classes[0]
    
        pred = pred.lower().capitalize()
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'LLaMA 70B 5 classes'] = pred
    except:
        print(el)

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_146048\233026555.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Consistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'LLaMA 70B 5 classes'] = pred


In [37]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

#### 3 classes

In [56]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_3_70B_instruct_3_types.pkl", 'rb') as f:
    answers_llama3_3_70B_instruct_3_types = pickle.load(f)

In [57]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in answers_llama3_3_70B_instruct_3_types.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_llama3_3_70B_instruct_3_types:
        answers_llama3_3_70B_instruct_3_types[new_key] = answers_llama3_3_70B_instruct_3_types.pop(old_key)

In [58]:
len(answers_llama3_3_70B_instruct_3_types)

698

In [66]:
df_gold['LLaMA 70B 3 classes'] = 0

In [60]:
classes_3_extended = classes_3 + classes_5 + ["Surface Contradiction", "Factual Inconsistency", "Indirect Inconsistency"]

In [61]:
to_edit = []
for el in answers_llama3_3_70B_instruct_3_types:
    res = answers_llama3_3_70B_instruct_3_types[el].content
    found_classes = [cls for cls in classes_3_extended if cls in res]
    # if not found_classes:
    #     print("Not found!")
    #     print(el)
    #     print(res)
    #     el = el.replace('\n\n', '\n')
    #     to_edit.append(el)
        
    # if len(set(found_classes))>1:

    match = re.search(r"Label:\s*(.*?)\n", res)
    # Extract and print the label if found
    if match:
        label = match.group(1) 
        # print(label)
    else:
        if not found_classes:
            print("No classes found")
            print(el)
            print(res)
            el = el.replace('\n\n', '\n')
            to_edit.append(el)
        elif len(set(found_classes)) > 1:
            if found_classes[0] != "Inconsistent":
                print("Too many classes")
                print(el)
                print(res)
                print(found_classes)
                el = el.replace('\n\n', '\n')
                to_edit.append(el)


In [62]:
wrong_to_correct_label_3_types = {"Indirect inconsistency": "Inconsistent", "Indirect Inconsistency": "Inconsistent",
                                  "Factual inconsistency": "Inconsistent", "Factual Inconsistency": "Inconsistent",
                                  "Surface contradiction": "Inconsistent", "Surface Contradiction": "Inconsistent", 
                                'Unrelated': 'Unrelated', 'Consistent': 'Consistent',
                                   'Inconsistent':  'Inconsistent'} 

In [67]:
predictions = []
true_labels = []
to_edit = []

for el in answers_llama3_3_70B_instruct_3_types:
    try:
        
        true_label = df_gold[df_gold['Input'] == el]['Answer_3_classes'].iloc[0]
        true_labels.append(true_label)
        
        res = answers_llama3_3_70B_instruct_3_types[el].content
        found_classes = [cls for cls in classes_3_extended if cls in res]
            
        found_classes = [wrong_to_correct_label_3_types[el] for el in found_classes]
        if len(set(found_classes)) > 1:
            print(found_classes)
            print(el)
            print(res)
            el = el.replace("\n\n", "\n")
            to_edit.append(el)
        pred = found_classes[0]
        
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'LLaMA 70B 3 classes'] = pred
    except:
        print("Some other error!")
        
        pass
        print(el)

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_146048\3450455261.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Consistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'LLaMA 70B 3 classes'] = pred


In [69]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

### Evaluate LLaMa 8B

#### 5 classes

In [77]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_8B_instruct_5_types.pkl", 'rb') as f:
    answers_llama3_8B_instruct_5_types = pickle.load(f)

In [78]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in answers_llama3_8B_instruct_5_types.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_llama3_8B_instruct_5_types:
        answers_llama3_8B_instruct_5_types[new_key] = answers_llama3_8B_instruct_5_types.pop(old_key)

In [48]:
len(answers_llama3_8B_instruct_5_types)

698

In [86]:
df_gold['LLaMA 5 classes'] = 0

In [89]:
predictions = []
true_labels = []
classes_5_casefold = classes_5 + ["Surface Contradiction", "Factual Inconsistency", "Indirect Inconsistency", "Related"]

for el in answers_llama3_8B_instruct_5_types:
    try:
        
        true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
        true_labels.append(true_label)
        
        res = answers_llama3_8B_instruct_5_types[el].content
        res = res.replace("Surface contraction", "Surface contradiction")
        found_classes = [cls for cls in classes_5_casefold if cls in res]
        if len(found_classes) > 1:
            print(found_classes)
        pred = found_classes[0]
        if pred == "Related":
            pred = "Unrelated"
        pred = pred.lower().capitalize()
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'LLaMA 5 classes'] = pred
    except:
        print(el)

In [90]:
Counter(df_gold['LLaMA 5 classes'])

Counter({'Factual inconsistency': 224,
         'Consistent': 161,
         'Unrelated': 145,
         'Indirect inconsistency': 85,
         'Surface contradiction': 83})

In [91]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

In [81]:
classes_5_casefold = classes_5 + ["Surface Contradiction", "Factual Inconsistency", "Indirect Inconsistency", "Related"]
for el in answers_llama3_8B_instruct_5_types:
    res = answers_llama3_8B_instruct_5_types[el].content
    found_classes = [cls for cls in classes_5_casefold if cls in res]
    if len(found_classes)>1:
        print(res)
        print(el)

#### 3 classes

In [60]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_8B_instruct_3_types.pkl", 'rb') as f:
    answers_llama3_8B_instruct_3_types = pickle.load(f)

In [61]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in answers_llama3_8B_instruct_3_types.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_llama3_8B_instruct_3_types:
        answers_llama3_8B_instruct_3_types[new_key] = answers_llama3_8B_instruct_3_types.pop(old_key)

In [62]:
len(answers_llama3_8B_instruct_3_types)

698

In [57]:
df_gold['LLaMA 3 classes'] = 0

In [64]:
df_gold.columns

Index(['Input', 'Answer', 'Confidence', 'All answers', 'Answer_3_classes',
       'ChatGPT-4 5 classes', 'ChatGPT-4 3 classes',
       'Model_agreement_with_human', 'model_agreement_with_human_3_classes',
       'ChatGPT-3.5 5 classes', 'ChatGPT-3.5 3 classes', 'LLaMA 5 classes',
       'LLaMA 3 classes'],
      dtype='object')

In [73]:
predictions = []
true_labels = []
for el in answers_llama3_8B_instruct_3_types:
    try:
        
        true_label = df_gold[df_gold['Input'] == el]['Answer_3_classes'].iloc[0]
        true_labels.append(true_label)
        
        res = answers_llama3_8B_instruct_3_types[el].content
        found_classes = [cls for cls in classes_3 if cls in res]
        
        pred = found_classes[0]
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'LLaMA 3 classes'] = pred
    except:
        print(el)

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_138736\169674657.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Consistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'LLaMA 3 classes'] = pred


In [68]:
for el in answers_llama3_8B_instruct_3_types:
    res = answers_llama3_8B_instruct_3_types[el].content
    found_classes = [cls for cls in classes_3 if cls in res]
    if len(found_classes) > 1:
        print(found_classes)
        print(res)
     

['Unrelated', 'Consistent', 'Inconsistent']
Label: Unrelated
Explanation: The Truth of Text 1 doesn't affect the Truth of Text 2, and vice versa. Text 1 talks about the importance of tourism for economic growth, while Text 2 discusses the potential impact of visa-free travel on national security. There is no logical connection between the two statements that would make them Consistent or Inconsistent.


In [74]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

### Eval ChatGPT-3.5 turbo, 5 types

In [3]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt35_turbo_5_types.pkl', 'rb') as f:
    chatgpt = pickle.load(f)

In [8]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in chatgpt.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in chatgpt:
        chatgpt[new_key] = chatgpt.pop(old_key)

In [12]:
len(chatgpt)

698

In [13]:
df_gold['ChatGPT-3.5 5 classes'] = 0

In [14]:
predictions = []
true_labels = []
for el in chatgpt:
    try:
        true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred = chatgpt[el]['label']
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'ChatGPT-3.5 5 classes'] = pred
    except:
        print(el)

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_136304\2737622744.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Factual inconsistency' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'ChatGPT-3.5 5 classes'] = pred


### Eval ChatGPT-3.5 turbo, 3 types

In [16]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt35_turbo_3_types.pkl', 'rb') as f:
    chatgpt = pickle.load(f)

In [20]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in chatgpt.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in chatgpt:
        chatgpt[new_key] = chatgpt.pop(old_key)

In [21]:
df_gold['ChatGPT-3.5 3 classes'] = 0

predictions = []
true_labels = []
for el in chatgpt:
    try:
        true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred = chatgpt[el]['label']
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'ChatGPT-3.5 3 classes'] = pred
    except:
        print(el)

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_136304\110831396.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'ChatGPT-3.5 3 classes'] = pred


Correct wrongly spelled labels

In [32]:
Counter(df_gold['ChatGPT-3.5 3 classes'])

Counter({'Inconsistent': 585, 'Consistent': 85, 'Unrelated': 28})

In [30]:
wrong_to_correct_label_3_types = {"Indirect inconsistency": "Inconsistent", "Factual inconsistency": "Inconsistent",
                                  "******** ": "Inconsistent", 'Unrelated': 'Unrelated', 'Consistent': 'Consistent',
                                   'Inconsistent':  'Inconsistent'} 

In [31]:
df_gold['ChatGPT-3.5 3 classes'] = list(map(lambda x: wrong_to_correct_label_3_types[x], df_gold['ChatGPT-3.5 3 classes']))

In [27]:
df_gold['ChatGPT-3.5 3 classes']=="******** "]['Input'].iloc[0]

'Text 1: The alignment of social standards and living conditions across the country is an important foundation of democratic structures.\n\nText 2: Financially strong states should not be required to support weaker states.'

In [ ]:
Label: Inconsistent Explanation: The first text emphasizes the alignment of social standards and living conditions across the country as a crucial aspect for democratic structures. In contrast, the second text suggests that financially strong states should not support weaker states, indicating a lack of alignment in economic conditions between states. This inconsistency arises from the different stances on the financial assistance between states, thereby contradicting the idea of social standards and living conditions alignment mentioned in the first text.

In [24]:
Counter(df_gold['ChatGPT-3.5 5 classes'])

Counter({'Factual inconsistency': 311,
         'Surface contradiction': 245,
         'Indirect inconsistency': 96,
         'Consistent': 36,
         'Unrelated': 10})

In [33]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

In [34]:
df_gold

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,Model_agreement_with_human,model_agreement_with_human_3_classes,ChatGPT-3.5 5 classes,ChatGPT-3.5 3 classes
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,1.000000,Factual inconsistency,Inconsistent
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,0.800000,Surface contradiction,Inconsistent
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,1.000000,Factual inconsistency,Inconsistent
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,Indirect inconsistency,Unrelated,0.200000,0.600000,Indirect inconsistency,Inconsistent
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,Indirect inconsistency,Inconsistent,0.833333,0.833333,Factual inconsistency,Inconsistent
...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Inconsistent,0.400000,1.000000,Surface contradiction,Inconsistent
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,1.000000,Surface contradiction,Inconsistent
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,0.800000,Surface contradiction,Inconsistent
696,Text 1: The level of social basic security mus...,Factual inconsistency,40.000,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,0.600000,Surface contradiction,Inconsistent


### Eval ChatGPT-4, 5 types

In [2]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_5_types.pkl', 'rb') as f:
    chatgpt_1 = pickle.load(f)
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_5_types_350_700.pkl', 'rb') as f:
    chatgpt_2 = pickle.load(f)

In [9]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in chatgpt_2.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in chatgpt_2:
        chatgpt_2[new_key] = chatgpt_2.pop(old_key)

In [12]:
chatgpt_all = dict()
chatgpt_all.update(chatgpt_1)
chatgpt_all.update(chatgpt_2)

In [17]:
df_gold['ChatGPT-4 5 classes'] = 0

In [18]:
predictions = []
true_labels = []
for el in chatgpt_all:
    try:
        true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred = chatgpt_all[el]['label']
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 5 classes'] = pred
    except:
        print(el)
    
# for el in chatgpt_2:
#     pred = chatgpt_2[el]['label']
#     predictions.append(pred)
#     true_label = df_gold[df_gold['Input'] == el]['Answer'].iloc[0]
#     true_labels.append(true_label)
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_119332\1598067405.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Surface contradiction' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 5 classes'] = pred


In [19]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_3_types.pkl', 'rb') as f:
    answers_chatgpt_3_types = pickle.load(f)

In [22]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt_3_types_350_700.pkl', 'rb') as f:
    answers_chatgpt_3_types_pt2 = pickle.load(f)

In [83]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt4_3_types_leftovers.pkl', 'rb') as f:
    answers_chatgpt4_3_types_leftovers = pickle.load(f)

In [25]:
### replace "\n" with "\n\n"
key_changes = dict()
for el in answers_chatgpt_3_types_pt2.keys():
    key_changes[el] = el.replace("\n", "\n\n")
    
# Rename keys
for old_key, new_key in key_changes.items():
    if old_key in answers_chatgpt_3_types_pt2:
        answers_chatgpt_3_types_pt2[new_key] = answers_chatgpt_3_types_pt2.pop(old_key)

In [27]:
answers_chatgpt_3_types_all = dict()
answers_chatgpt_3_types_all.update(answers_chatgpt_3_types)
answers_chatgpt_3_types_all.update(answers_chatgpt_3_types_pt2)

In [85]:
answers_chatgpt_3_types_all.update(answers_chatgpt4_3_types_leftovers)

In [87]:
df_gold['ChatGPT-4 3 classes'] = 0
predictions = []
true_labels = []
for el in answers_chatgpt_3_types_all:
    try:
        true_label = df_gold[df_gold['Input'] == el]['Answer_3_classes'].iloc[0]
        true_labels.append(true_label)
        pred = answers_chatgpt_3_types_all[el]['label']
        predictions.append(pred)
        df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 3 classes'] = pred
    except:
        print(el)
    

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_119332\658612249.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Surface contradiction' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold.loc[df_gold['Input'] == el, 'ChatGPT-4 3 classes'] = pred


In [48]:
wrong_to_correct_label_5_types = {"Factually inconsistent": "Factual inconsistency", 
                                  'Unrelated': 'Unrelated', 'Consistent': 'Consistent',
                                  'Surface contradiction': 'Surface contradiction',
                                  'Factual inconsistency': 'Factual inconsistency',
                                  'Indirect inconsistency': 'Indirect inconsistency'
                                  }
 
wrong_to_correct_label_3_types = {"Indirect inconsistency": "Inconsistent", "Factual inconsistency": "Inconsistent",
                                  "Surface contradiction": "Inconsistent", 'Unrelated': 'Unrelated', 'Consistent': 'Consistent',
                                   'Inconsistent':  'Inconsistent',
                                   'Indirect Inconsistency': 'Inconsistent'}
 

In [89]:
Counter(df_gold['ChatGPT-4 3 classes'])

Counter({'Inconsistent': 189,
         'Indirect inconsistency': 185,
         'Unrelated': 138,
         'Consistent': 130,
         'Surface contradiction': 32,
         'Factual inconsistency': 21,
         'Indirect Inconsistency': 3})

In [92]:
df_gold['ChatGPT-4 3 classes'] = list(map(lambda x: wrong_to_correct_label_3_types[x], df_gold['ChatGPT-4 3 classes']))

In [61]:
df_gold['ChatGPT-4 5 classes'] = list(map(lambda x: wrong_to_correct_label_5_types[x], df_gold['ChatGPT-4 5 classes']))

In [164]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

Format answers so that they are not misread

### Accuracy Part 1 ChatGPT-4 

5 classes is 0.5374

3 classes with different prompt Accuracy is 0.2471. It probably misunderstood the assignment, and even tho it was asked to output 3 classes, it did 5.

If we aggregate over classes: 
Accuracy: 0.8563

### Accuracy Part 1 ChatGPT-3.5 

5 classes is 0.3724 - 341 samples (TODO: fix parsing)

If we aggregate over classes: 
Accuracy: 0.7040

3 classes prompt:
Accuracy: 0.1026


### Accuracy Part 1 Llama3.3 70B Instruct 

5 classes is 0.4751 - 341 samples (TODO: fix parsing)

If aggregared: 
Accuracy: 0.7989

3 classes prompt: Accuracy: 0.1554


### Accuracy Part 1 Llama3 8B Instruct

5 classes: 0.3255 - 341 samples (TODO: fix parsing)

If aggregated: Accuracy: 0.5747

### Accuracy Part 1 Mixtral_8_7B_Instruct_v0_1t

5 classes: 0.0176

If aggregated: 0.0172




### Measure accuracy

In [98]:
true_labels = list(df_gold['Answer_3_classes'])
predictions = list(df_gold['ChatGPT-4 3 classes'])

In [99]:
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.8009


##### Per class accuracy

In [4]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             
classes_3 = ['Unrelated', 'Consistent', 'Inconsistent']

In [5]:
model_acc_per_class_f1_score = dict()

In [8]:
model_acc_per_class = dict()

In [9]:
model_acc_per_class_f1_score['chatgpt4'] = {}

model_acc_per_class_f1_score['chatgpt4']['5_classes'] = dict()
for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['ChatGPT-4 5 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"F1 Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['chatgpt4']['5_classes'][class_name] = accuracy
    

Unrelated
F1 Accuracy: 0.2005
Consistent
F1 Accuracy: 0.3131
Indirect inconsistency
F1 Accuracy: 0.2085
Factual inconsistency
F1 Accuracy: 0.0587
Surface contradiction
F1 Accuracy: 0.0988


In [10]:
true_labels = list(df_gold['Answer'])
predictions = list(df_gold['ChatGPT-4 5 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['chatgpt4']['5_classes']['total'] = accuracy

F1 Accuracy: 0.5364


In [97]:
model_acc_per_class['chatgpt4'] = {}

model_acc_per_class['chatgpt4']['5_classes'] = dict()
for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['ChatGPT-4 5 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['chatgpt4']['5_classes'][class_name] = accuracy
    

Unrelated
Accuracy: 0.6695
Consistent
Accuracy: 0.8854
Indirect inconsistency
Accuracy: 0.7151
Factual inconsistency
Accuracy: 0.1721
Surface contradiction
Accuracy: 0.3279


In [11]:
model_acc_per_class_f1_score['chatgpt4']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['ChatGPT-4 3 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['chatgpt4']['3_classes'][class_name] = accuracy
    
true_labels = list(df_gold['Answer_3_classes'])
predictions = list(df_gold['ChatGPT-4 3 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['chatgpt4']['3_classes']['total'] = accuracy

Unrelated
Accuracy: 0.2772
Consistent
Accuracy: 0.3051
Inconsistent
Accuracy: 0.2992
F1 Accuracy: 0.7451


In [99]:
model_acc_per_class['chatgpt4']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['ChatGPT-4 3 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['chatgpt4']['3_classes'][class_name] = accuracy

Unrelated
Accuracy: 0.7119
Consistent
Accuracy: 0.8438
Inconsistent
Accuracy: 0.8140


In [12]:
model_acc_per_class_f1_score['chatgpt_3.5_turbo'] = {}

model_acc_per_class_f1_score['chatgpt_3.5_turbo']['5_classes'] = dict()

for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['ChatGPT-3.5 5 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['chatgpt_3.5_turbo']['5_classes'][class_name] = accuracy
    
    
true_labels = list(df_gold['Answer'])
predictions = list(df_gold['ChatGPT-3.5 5 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['chatgpt_3.5_turbo']['5_classes']['total'] = accuracy

model_acc_per_class_f1_score['chatgpt_3.5_turbo']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['ChatGPT-3.5 3 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['chatgpt_3.5_turbo']['3_classes'][class_name] = accuracy
    
true_labels = list(df_gold['Answer_3_classes'])
predictions = list(df_gold['ChatGPT-3.5 3 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['chatgpt_3.5_turbo']['3_classes']['total'] = accuracy

Unrelated
Accuracy: 0.0312
Consistent
Accuracy: 0.1160
Indirect inconsistency
Accuracy: 0.0856
Factual inconsistency
Accuracy: 0.1965
Surface contradiction
Accuracy: 0.1709
F1 Accuracy: 0.3347
Unrelated
Accuracy: 0.1007
Consistent
Accuracy: 0.2456
Inconsistent
Accuracy: 0.3249
F1 Accuracy: 0.5890


In [100]:
model_acc_per_class['chatgpt_3.5_turbo'] = {}

model_acc_per_class['chatgpt_3.5_turbo']['5_classes'] = dict()

for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['ChatGPT-3.5 5 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['chatgpt_3.5_turbo']['5_classes'][class_name] = accuracy
    
    
model_acc_per_class['chatgpt_3.5_turbo']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['ChatGPT-3.5 3 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['chatgpt_3.5_turbo']['3_classes'][class_name] = accuracy

Unrelated
Accuracy: 0.0847
Consistent
Accuracy: 0.3021
Indirect inconsistency
Accuracy: 0.2067
Factual inconsistency
Accuracy: 0.6475
Surface contradiction
Accuracy: 0.5191
Unrelated
Accuracy: 0.1780
Consistent
Accuracy: 0.5833
Inconsistent
Accuracy: 0.9504


In [13]:
model_acc_per_class_f1_score['LLaMA 8B'] = {}

model_acc_per_class_f1_score['LLaMA 8B']['5_classes'] = dict()

for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['LLaMA 5 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['LLaMA 8B']['5_classes'][class_name] = accuracy
    
true_labels = list(df_gold['Answer'])
predictions = list(df_gold['LLaMA 5 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['LLaMA 8B']['5_classes']['total'] = accuracy

    
model_acc_per_class_f1_score['LLaMA 8B']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['LLaMA 3 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['LLaMA 8B']['3_classes'][class_name] = accuracy
    
true_labels = list(df_gold['Answer_3_classes'])
predictions = list(df_gold['LLaMA 3 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['LLaMA 8B']['3_classes']['total'] = accuracy

Unrelated
Accuracy: 0.1240
Consistent
Accuracy: 0.1673
Indirect inconsistency
Accuracy: 0.0558
Factual inconsistency
Accuracy: 0.1289
Surface contradiction
Accuracy: 0.0688
F1 Accuracy: 0.3561
Unrelated
Accuracy: 0.1558
Consistent
Accuracy: 0.2880
Inconsistent
Accuracy: 0.2857
F1 Accuracy: 0.5653


In [105]:
model_acc_per_class['LLaMA 8B'] = {}

model_acc_per_class['LLaMA 8B']['5_classes'] = dict()

for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['LLaMA 5 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['LLaMA 8B']['5_classes'][class_name] = accuracy
    
    
model_acc_per_class['LLaMA 8B']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['LLaMA 3 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['LLaMA 8B']['3_classes'][class_name] = accuracy

Unrelated
Accuracy: 0.4492
Consistent
Accuracy: 0.7188
Indirect inconsistency
Accuracy: 0.1620
Factual inconsistency
Accuracy: 0.4754
Surface contradiction
Accuracy: 0.2077
Unrelated
Accuracy: 0.3051
Consistent
Accuracy: 0.7604
Inconsistent
Accuracy: 0.7500


In [14]:
model_acc_per_class_f1_score['LLaMA 70B'] = {}

model_acc_per_class_f1_score['LLaMA 70B']['5_classes'] = dict()

for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['LLaMA 70B 5 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['LLaMA 70B']['5_classes'][class_name] = accuracy
    
true_labels = list(df_gold['Answer'])
predictions = list(df_gold['LLaMA 70B 5 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['LLaMA 70B']['5_classes']['total'] = accuracy

    
model_acc_per_class_f1_score['LLaMA 70B']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['LLaMA 70B 3 classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['LLaMA 70B']['3_classes'][class_name] = accuracy
    
true_labels = list(df_gold['Answer_3_classes'])
predictions = list(df_gold['LLaMA 70B 3 classes'])
accuracy = f1_score(true_labels, predictions, average='macro')
print(f"F1 Accuracy: {accuracy:.4f}")
model_acc_per_class_f1_score['LLaMA 70B']['3_classes']['total'] = accuracy

Unrelated
Accuracy: 0.1122
Consistent
Accuracy: 0.2193
Indirect inconsistency
Accuracy: 0.1769
Factual inconsistency
Accuracy: 0.0634
Surface contradiction
Accuracy: 0.1107
F1 Accuracy: 0.5021
Unrelated
Accuracy: 0.1899
Consistent
Accuracy: 0.3030
Inconsistent
Accuracy: 0.3152
F1 Accuracy: 0.7038


In [106]:
model_acc_per_class['LLaMA 70B'] = {}

model_acc_per_class['LLaMA 70B']['5_classes'] = dict()

for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    true_labels = list(df_class['Answer'])
    predictions = list(df_class['LLaMA 70B 5 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['LLaMA 70B']['5_classes'][class_name] = accuracy
    
    
model_acc_per_class['LLaMA 70B']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['LLaMA 70B 3 classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['LLaMA 70B']['3_classes'][class_name] = accuracy

Unrelated
Accuracy: 0.3898
Consistent
Accuracy: 0.7812
Indirect inconsistency
Accuracy: 0.7933
Factual inconsistency
Accuracy: 0.1885
Surface contradiction
Accuracy: 0.3825
Unrelated
Accuracy: 0.3983
Consistent
Accuracy: 0.8333
Inconsistent
Accuracy: 0.8967


In [15]:
model_acc_per_class_f1_score['DeBERTa NLI'] = {}

model_acc_per_class_f1_score['DeBERTa NLI']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['deberta-v3-large-nli_3_classes'])
    accuracy = f1_score(true_labels, predictions, average='macro')
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class_f1_score['DeBERTa NLI']['3_classes'][class_name] = accuracy

Unrelated
Accuracy: 0.0056
Consistent
Accuracy: 0.3924
Inconsistent
Accuracy: 0.4812


In [108]:
model_acc_per_class['DeBERTa NLI'] = {}

model_acc_per_class['DeBERTa NLI']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    true_labels = list(df_class['Answer_3_classes'])
    predictions = list(df_class['deberta-v3-large-nli_3_classes'])
    accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
    print(f"Accuracy: {accuracy:.4f}")
    model_acc_per_class['DeBERTa NLI']['3_classes'][class_name] = accuracy

Unrelated
Accuracy: 0.0085
Consistent
Accuracy: 0.6458
Inconsistent
Accuracy: 0.9277


In [16]:
# with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_acc_per_class_f1_score.pkl", 'wb') as f:
#     pickle.dump(model_acc_per_class_f1_score, f)

In [109]:
# with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_acc_per_class.pkl", 'wb') as f:
#     pickle.dump(model_acc_per_class, f)

#### Pretend model is 6th annotator

In [112]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_agreement_per_class.pkl", 'rb') as f:
    model_agreement_per_class = pickle.load(f)

In [110]:
label_to_num = {'Unrelated': 1, "Consistent": 2, "Indirect inconsistency": 3, "Factual inconsistency": 4, "Surface contradiction": 5}

In [111]:
label_to_num = {'Unrelated': 1, "Consistent": 2,  "Inconsistent": 3, "Indirect inconsistency": 3, "Factual inconsistency": 3, "Surface contradiction": 3}

In [114]:
with open("all_sample_to_responses_some_6.pkl", 'rb') as f:
    final_sample_to_responses = pickle.load(f)

In [38]:
for el in final_sample_to_responses:
    model_label = df_gold[df_gold['Input'] == el]['ChatGPT-3.5 3 classes'].iloc[0]
    final_sample_to_responses[el].append(model_label)

In [39]:
matrix = []
# here have to drop 1 sample for which 10 annotations were received accidentally
for sample in final_sample_to_responses:
    res = []
    for answer in final_sample_to_responses[sample]:
        res.append(label_to_num[answer])
    if len(res) == 6:
        matrix.append(res)
    else:
        print(sample)
        print(res)
        matrix.append(random.sample(res, 6))


Text 1: We fully support the conversion of additional car streets to bicycle streets.

Text 2: We propose reducing the taxes on the purchase of new cars to stimulate the automotive industry
[1, 3, 3, 3, 3, 3, 3]
Text 1: The UN should have more power to enforce resolutions.

Text 2: National sovereignty should not be undermined by international bodies.
[3, 3, 3, 3, 3, 3, 3]
Text 1: The extraction of gas from rock layers (fracking) is to be completely banned.

Text 2: Fracking poses a potential threat to groundwater.
[2, 2, 2, 2, 2, 2, 2]
Text 1: There should be a general vaccination requirement against Covid-19.

Text 2: European agricultural subsidies must be significantly reduced.
[1, 1, 1, 1, 1, 1, 1]
Text 1: To combat terrorism, the armed forces should also be deployable domestically.

Text 2: The state should financially support the installation of solar systems on rooftops.
[1, 1, 1, 1, 1, 1, 1]
Text 1: The surveillance state is undemocratic and is meant to intimidate people.

Tex

In [40]:
matrix = np.array(matrix)

matrix_fleiss = []
for row in matrix:
    res = []
    counts = Counter(row)
    for i in range(1, 4): # 6 if all labels, 4 if merged
        res.append(counts[i])
    matrix_fleiss.append(res)

In [41]:
matrix_fleiss = np.array(matrix_fleiss)

print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=matrix_fleiss,
                                                                          level_of_measurement='nominal'))
print("Krippendorff's alpha for interval metric: ", krippendorff.alpha(value_counts=matrix_fleiss))

Krippendorff's alpha for nominal metric:  0.4668612029495651
Krippendorff's alpha for interval metric:  0.419078631516941


#### What is a probability that model agrees with indiv. (model - random human for each question)

for this question it agrees with 3/6 annotators -> avg agreement

per question, per class, overall


In [123]:
model_agreement_with_human = []
for i in range(len(df_gold)):
    human_answers = eval(df_gold.iloc[i]['All answers'])
    model_answer = df_gold.iloc[i]['LLaMA 70B 5 classes']
    model_agreement = Counter(human_answers)[model_answer]/len(human_answers)
    model_agreement_with_human.append(model_agreement)
    
df_gold['Model_agreement_with_human_LLaMA_70B'] = model_agreement_with_human

model_agreement_with_human_3_classes = []
for i in range(len(df_gold)):
    human_answers = eval(df_gold.iloc[i]['All answers'])
    human_answers = [wrong_to_correct_label_3_types[el] for el in human_answers]
    model_answer = df_gold.iloc[i]['LLaMA 70B 3 classes']
    model_agreement = Counter(human_answers)[model_answer]/len(human_answers)
    model_agreement_with_human_3_classes.append(model_agreement)
    
df_gold['Model_agreement_with_human_3_classes_LLaMA_70B'] = model_agreement_with_human_3_classes

In [126]:
model_agreement_with_human_3_classes = []
for i in range(len(df_gold)):
    human_answers = eval(df_gold.iloc[i]['All answers'])
    human_answers = [wrong_to_correct_label_3_types[el] for el in human_answers]
    model_answer = df_gold.iloc[i]['deberta-v3-large-nli_3_classes']
    model_agreement = Counter(human_answers)[model_answer]/len(human_answers)
    model_agreement_with_human_3_classes.append(model_agreement)
    
df_gold['Model_agreement_with_human_3_classes_deberta_nli'] = model_agreement_with_human_3_classes

In [128]:
df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

#### Per class agreement with humans

In [52]:
model_agreement_per_class = dict()

In [54]:
model_agreement_per_class['chatgpt_4'] = dict()
model_agreement_per_class['chatgpt_4']['3_classes'] = dict()
model_agreement_per_class['chatgpt_4']['5_classes'] = dict()

model_agreement_per_class['chatgpt_4']['5_classes']['Unrelated'] = 0.5932203389830506
model_agreement_per_class['chatgpt_4']['5_classes']['Consistent'] = 0.7413194444444442
model_agreement_per_class['chatgpt_4']['5_classes']['Indirect inconsistency'] = 0.49273743016759736
model_agreement_per_class['chatgpt_4']['5_classes']['Factual inconsistency'] = 0.26530054644808726
model_agreement_per_class['chatgpt_4']['5_classes']['Surface contradiction'] = 0.36630236794171245


model_agreement_per_class['chatgpt_4']['3_classes']['Unrelated'] = 0.6703389830508473
model_agreement_per_class['chatgpt_4']['3_classes']['Consistent'] = 0.740972222222222
model_agreement_per_class['chatgpt_4']['3_classes']['Inconsistent'] = 0.7789256198347121

In [69]:
# with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_agreement_per_class.pkl", 'wb') as f:
#     pickle.dump(model_agreement_per_class, f)

In [66]:
model_agreement_per_class['chatgpt_3.5_turbo'] = dict()
model_agreement_per_class['chatgpt_3.5_turbo']['5_classes'] = dict()
for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    avg_agreement = sum(list(df_class['Model_agreement_with_human_gpt3.5_turbo']))/len(df_class)
    print(avg_agreement)
    model_agreement_per_class['chatgpt_3.5_turbo']['5_classes'][class_name] = avg_agreement

Unrelated
0.13220338983050844
Consistent
0.3270833333333333
Indirect inconsistency
0.23258845437616382
Factual inconsistency
0.4489071038251368
Surface contradiction
0.45628415300546477


In [67]:
model_agreement_per_class['chatgpt_3.5_turbo']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    avg_agreement = sum(list(df_class['model_agreement_with_human_3_classes_gpt3.5_turbo']))/len(df_class)
    print(avg_agreement)
    model_agreement_per_class['chatgpt_3.5_turbo']['3_classes'][class_name] = avg_agreement

Unrelated
0.3779661016949154
Consistent
0.6055555555555556
Inconsistent
0.865220385674933


In [125]:
model_agreement_per_class

{'chatgpt_4': {'3_classes': {'Unrelated': 0.6703389830508473,
   'Consistent': 0.740972222222222,
   'Inconsistent': 0.7789256198347121},
  '5_classes': {'Unrelated': 0.5932203389830506,
   'Consistent': 0.7413194444444442,
   'Indirect inconsistency': 0.49273743016759736,
   'Factual inconsistency': 0.26530054644808726,
   'Surface contradiction': 0.36630236794171245}},
 'chatgpt_3.5_turbo': {'5_classes': {'Unrelated': 0.13220338983050844,
   'Consistent': 0.3270833333333333,
   'Indirect inconsistency': 0.23258845437616382,
   'Factual inconsistency': 0.4489071038251368,
   'Surface contradiction': 0.45628415300546477},
  '3_classes': {'Unrelated': 0.3779661016949154,
   'Consistent': 0.6055555555555556,
   'Inconsistent': 0.865220385674933}}}

In [131]:
model_agreement_per_class['LLaMA 8B'] = dict()
model_agreement_per_class['LLaMA 8B']['5_classes'] = dict()
for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    avg_agreement = sum(list(df_class['Model_agreement_with_human_LLaMA_8B']))/len(df_class)
    print(avg_agreement)
    model_agreement_per_class['LLaMA 8B']['5_classes'][class_name] = avg_agreement

model_agreement_per_class['LLaMA 8B']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    avg_agreement = sum(list(df_class['Model_agreement_with_human_3_classes_LLaMA_8B']))/len(df_class)
    print(avg_agreement)
    model_agreement_per_class['LLaMA 8B']['3_classes'][class_name] = avg_agreement

Unrelated
0.411864406779661
Consistent
0.6045138888888889
Indirect inconsistency
0.21229050279329598
Factual inconsistency
0.34562841530054667
Surface contradiction
0.2548269581056466
Unrelated
0.4135593220338984
Consistent
0.6972222222222221
Inconsistent
0.7141873278236924


In [132]:
model_agreement_per_class['LLaMA 70B'] = dict()
model_agreement_per_class['LLaMA 70B']['5_classes'] = dict()
for class_name in classes_5:
    print(class_name)
    df_class = df_gold[df_gold['Answer'] == class_name]
    avg_agreement = sum(list(df_class['Model_agreement_with_human_LLaMA_70B']))/len(df_class)
    print(avg_agreement)
    model_agreement_per_class['LLaMA 70B']['5_classes'][class_name] = avg_agreement

model_agreement_per_class['LLaMA 70B']['3_classes'] = dict()
for class_name in classes_3:
    print(class_name)
    df_class = df_gold[df_gold['Answer_3_classes'] == class_name]
    avg_agreement = sum(list(df_class['Model_agreement_with_human_3_classes_LLaMA_70B']))/len(df_class)
    print(avg_agreement)
    model_agreement_per_class['LLaMA 70B']['3_classes'][class_name] = avg_agreement

Unrelated
0.4466101694915253
Consistent
0.689236111111111
Indirect inconsistency
0.5091247672253254
Factual inconsistency
0.2898907103825136
Surface contradiction
0.3992714025500912
Unrelated
0.5203389830508476
Consistent
0.7493055555555553
Inconsistent
0.8346418732782382


In [135]:
# with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_agreement_per_class.pkl", 'wb') as f:
#     pickle.dump(model_agreement_per_class, f)

#### Performance in comparison to humans

In [106]:
df_gold_part_1['ChatGPT-4 5 classes aggregated'] = 0

df_gold_part_1['ChatGPT-4 5 classes aggregated'] = df_gold_part_1['ChatGPT-4 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

In [116]:
predictions = list(df_gold_part_1["ChatGPT-4 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])

In [117]:
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8563


### Eval ChatGPT-3.5, 5 types

In [119]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt35_turbo_5_types.pkl', 'rb') as f:
    answers_chatgpt35_turbo_5_types = pickle.load(f)
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_chatgpt35_turbo_3_types.pkl', 'rb') as f:
    answers_chatgpt35_turbo_3_types = pickle.load(f)

In [120]:
df_gold_part_1['ChatGPT-3.5 5 classes'] = 0
df_gold_part_1['ChatGPT-3.5 3 classes'] = 0
df_gold_part_1['ChatGPT-3.5 5 classes aggregated'] = 0

In [134]:
el.split('\n')[0]

'Text 1: We support granting exceptions for clubs in the city center from nighttime noise restrictions.'

In [148]:
predictions = []
true_labels = []
count_fail = 0
for el in answers_chatgpt35_turbo_3_types:
    try:
        text1, text2 = el.split('\n')[0], el.split('\n')[-1]
        true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred = answers_chatgpt35_turbo_3_types[el]['label']
        predictions.append(pred)
        new_text = text1 + '\n\n' + text2
        df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'ChatGPT-3.5 3 classes'] = pred
    except:
        # print(el)
        count_fail += 1
        
        
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\3383750435.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Inconsistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'ChatGPT-3.5 3 classes'] = pred
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\3383750435.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]


Accuracy: 0.1026


In [146]:
df_gold_part_1['ChatGPT-3.5 5 classes aggregated'] = 0
df_gold_part_1['ChatGPT-3.5 5 classes aggregated'] = df_gold_part_1['ChatGPT-3.5 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

predictions = list(df_gold_part_1["ChatGPT-3.5 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7040


### Evaluate LLaMa

In [192]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_8B_instruct_5_types.pkl", 'rb') as f:
    answers_llama3_8B_instruct_5_types = pickle.load(f)

In [178]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_llama3_3_70B_instruct_3_types.pkl", 'rb') as f:
    answers_llama3_3_70B_instruct_3_types = pickle.load(f)

In [154]:
def extract_label_and_explanation(text):
    label_match = re.search(r"Label:(.*?)Explanation:", text, re.DOTALL)
    label_text = label_match.group(1).strip() if label_match else text

    # Extract text after "Explanation"
    explanation_match = re.search(r"Explanation:(.*)", text, re.DOTALL)
    explanation_text = explanation_match.group(1).strip() if explanation_match else text
    
    return label_text, explanation_text

In [193]:
df_gold_part_1['Llama3-8B-instruct 5 classes'] = 0
df_gold_part_1['Llama3-8B-instruct 3 classes'] = 0
df_gold_part_1['Llama3-8B-instruct 5 classes aggregated'] = 0

In [194]:
predictions = []
true_labels = []
count_fail = 0
for el in answers_llama3_8B_instruct_5_types:
    try:
        text1, text2 = el.split('\n')[0], el.split('\n')[-1]
        true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred, explanation = extract_label_and_explanation(str(answers_llama3_8B_instruct_5_types[el]))
        pred = pred.replace('\\n', '')
        predictions.append(pred)
        new_text = text1 + '\n\n' + text2
        df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Llama3-8B-instruct 5 classes'] = pred
    except:
        # print(el)
        count_fail += 1
        
        
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\2112758861.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Consistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Llama3-8B-instruct 5 classes'] = pred
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\2112758861.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]


Accuracy: 0.3255


In [195]:
df_gold_part_1['Llama3-8B-instruct 5 classes aggregated'] = 0
df_gold_part_1['Llama3-8B-instruct 5 classes aggregated'] = df_gold_part_1['Llama3-8B-instruct 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

predictions = list(df_gold_part_1["Llama3-8B-instruct 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.5747


### Evaluate Mistral

In [196]:
with open(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\notebooks\evaluating_models\answers_mixtral_8_7B_Instruct_v0_1t_5_types.pkl", 'rb') as f:
    answers_mixtral_8_7B_Instruct_v0_1t_5_types = pickle.load(f)

In [197]:
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes'] = 0
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 3 classes'] = 0
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated'] = 0

In [200]:
answers_mixtral_8_7B_Instruct_v0_1t_5_types

{"Text 1: The first degree should remain free of charge for everyone.\nText 2: Scholarship should be paid regardless of the parents' income.": ChatCompletionMessage(content=" Label: Consistent\n\nExplanation: Both statements support the idea that education should be accessible and affordable for everyone, regardless of their financial situation. The first statement suggests that education should be free of charge for everyone, while the second statement suggests that scholarships should be provided regardless of the parents' income. These statements are not contradictory, as they both advocate for making education more accessible and affordable.", role='assistant', function_call=None, tool_calls=None),
 'Text 1: Only states that consistently respect human rights should receive development aid.\nText 2: Trade relations with states that violate human rights should be stopped.': ChatCompletionMessage(content=' Label: Indirect inconsistency\nExplanation: The first statement implies that de

In [201]:
predictions = []
true_labels = []
count_fail = 0
for el in answers_mixtral_8_7B_Instruct_v0_1t_5_types:
    try:
        text1, text2 = el.split('\n')[0], el.split('\n')[-1]
        true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]
        true_labels.append(true_label)
        pred, explanation = extract_label_and_explanation(str(answers_mixtral_8_7B_Instruct_v0_1t_5_types[el]))
        pred = pred.replace('\\n', '')
        predictions.append(pred)
        new_text = text1 + '\n\n' + text2
        df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Mixtral_8_7B_Instruct_v0_1t 5 classes'] = pred
    except:
        # print(el)
        count_fail += 1
        
        
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\614373465.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Consistent' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_gold_part_1.loc[df_gold_part_1['Input'] == new_text, 'Mixtral_8_7B_Instruct_v0_1t 5 classes'] = pred
C:\Users\Nursulu_1\AppData\Local\Temp\ipykernel_154864\614373465.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  true_label = df_gold_part_1[df_gold_part_1['Input'].str.contains(text1) & df_gold_part_1['Input'].str.contains(text2)]['Answer'].iloc[0]


Accuracy: 0.0176


In [205]:
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated'] = 0
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated'] = df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes'].apply(lambda x: 'Inconsistent' if x in ['Surface contradiction', 'Factual inconsistency', 'Indirect inconsistency'] else x)

predictions = list(df_gold_part_1["Mixtral_8_7B_Instruct_v0_1t 5 classes aggregated"])
true_labels = list(df_gold_part_1["Answer_3_classes"])
accuracy = sum(p == t for p, t in zip(predictions, true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.0172


In [202]:
698 - count_fail

341

In [204]:
df_gold_part_1['Mixtral_8_7B_Instruct_v0_1t 5 classes']

0      ChatCompletionMessage(content='', role='assist...
1      ChatCompletionMessage(content=' (< {  Text 1 `...
2      ChatCompletionMessage(content=" The company's ...
3      ChatCompletionMessage(content='    Explanation...
4      ChatCompletionMessage(content=' Text 1: You ar...
                             ...                        
343    ChatCompletionMessage(content=' 1.gen``` AU. b...
344    ChatCompletionMessage(content=' Consistnt ````...
345    ChatCompletionMessage(content='```', role='ass...
346    ChatCompletionMessage(content='     Text:dAnsw...
347    ChatCompletionMessage(content='```', role='ass...
Name: Mixtral_8_7B_Instruct_v0_1t 5 classes, Length: 348, dtype: object

### Evaluate random human-human agreement

In [5]:
import itertools

In [13]:
human_human_agreement = []
for i in range(len(df_gold)):
    strings = eval(df_gold['All answers'][i])
    # Generate all unique pairs
    pairs = list(itertools.combinations(strings, 2))

    count_same = 0
    # Display the pairs
    for pair in pairs:
        if pair[0] == pair[1]:
            count_same += 1

    avg_agreement = count_same/len(pairs)
    human_human_agreement.append(avg_agreement)

In [15]:
df_gold['human_human_agreement'] = human_human_agreement

In [37]:
### Each individual VS model
chatgpt4_individuals_agreement = []
chatgpt3_5_individuals_agreement = []
llama_8b_individuals_agreement = []
llama_70b_individuals_agreement = []

for i in range(len(df_gold)):
    strings = eval(df_gold['All answers'][i])
    # Generate all unique pairs
    count_same_chatgpt4 = 0
    count_same_chatgpt3_5 = 0
    count_same_llama_8b = 0
    count_same_llama_70b = 0
    
    for el in strings:
        if df_gold['ChatGPT-4 5 classes'][i] == el:
            count_same_chatgpt4 += 1
        if df_gold['ChatGPT-3.5 5 classes'][i] == el:
            count_same_chatgpt3_5 += 1
        if df_gold['LLaMA 5 classes'][i] == el:
            count_same_llama_8b += 1
        if df_gold['LLaMA 70B 5 classes'][i] == el:
            count_same_llama_70b += 1

    avg_agreement_chatgpt4 = count_same_chatgpt4/len(strings)
    avg_agreement_chatgpt3_5 = count_same_chatgpt3_5/len(strings)
    avg_agreement_llama_8b = count_same_llama_8b/len(strings)
    avg_agreement_llama_70b = count_same_llama_70b/len(strings)
    
    chatgpt4_individuals_agreement.append(avg_agreement_chatgpt4)
    chatgpt3_5_individuals_agreement.append(avg_agreement_chatgpt3_5)
    llama_8b_individuals_agreement.append(avg_agreement_llama_8b)
    llama_70b_individuals_agreement.append(avg_agreement_llama_70b)

In [38]:
df_gold['chatgpt4_individuals_agreement'] = chatgpt4_individuals_agreement

df_gold['chatgpt3_5_individuals_agreement'] = chatgpt3_5_individuals_agreement

df_gold['llama_8b_individuals_agreement'] = llama_8b_individuals_agreement

df_gold['llama_70b_individuals_agreement'] = llama_70b_individuals_agreement


In [39]:
# df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

In [33]:
human_crowd_wisdom_agreement = []
chatgpt4_crowd_wisdom_agreement = []
chatgpt3_5_crowd_wisdom_agreement = []
llama_8b_crowd_wisdom_agreement = []
llama_70b_crowd_wisdom_agreement = []

for i in range(len(df_gold)):
    
    strings = eval(df_gold['All answers'][i])
    # create versions where we drop each value
    versions = [(strings[j], strings[:j] + strings[j+1:]) for j in range(len(strings))]

    # print(strings)
    count_agreed = 0
    chatgpt4_agreed = 0
    chatgpt3_5_agreed = 0
    llama_8b_agreed = 0
    llama_70b_agreed = 0
    
    # go through each element
    for value, version in versions:
        # print('dropping', value)
        list_annotator_dropped = version
        # print("list_annotator_dropped", list_annotator_dropped)
        counter = Counter(list_annotator_dropped)
        # identify the majority vote
        majority, count = counter.most_common(1)[0]
        # drop 1 person all the time
        # print(majority)
        # print(counter)
        if value == majority:
            count_agreed += 1
        if df_gold['ChatGPT-4 5 classes'][i] == majority:
            chatgpt4_agreed += 1
        if df_gold['ChatGPT-3.5 5 classes'][i] == majority:
            chatgpt3_5_agreed += 1
        if df_gold['LLaMA 5 classes'][i] == majority:
            llama_8b_agreed += 1
        if df_gold['LLaMA 70B 5 classes'][i] == majority:
            llama_70b_agreed += 1
            
    avg_agreed_with_majority = count_agreed/len(versions)
    avg_agreed_with_majority_chatgpt_4 = chatgpt4_agreed/len(versions)
    avg_agreed_with_majority_chatgpt_3_5 = chatgpt3_5_agreed/len(versions)
    avg_agreed_with_majority_llama_8b = llama_8b_agreed/len(versions)
    avg_agreed_with_majority_llama_70b = llama_70b_agreed/len(versions)

    human_crowd_wisdom_agreement.append(avg_agreed_with_majority)
    chatgpt4_crowd_wisdom_agreement.append(avg_agreed_with_majority_chatgpt_4)
    chatgpt3_5_crowd_wisdom_agreement.append(avg_agreed_with_majority_chatgpt_3_5)
    llama_8b_crowd_wisdom_agreement.append(avg_agreed_with_majority_llama_8b)
    llama_70b_crowd_wisdom_agreement.append(avg_agreed_with_majority_llama_70b)

In [30]:
df_gold['human_crowd_wisdom_agreement'] = human_crowd_wisdom_agreement

In [34]:
df_gold['chatgpt4_crowd_wisdom_agreement'] = chatgpt4_crowd_wisdom_agreement

df_gold['chatgpt3_5_crowd_wisdom_agreement'] = chatgpt3_5_crowd_wisdom_agreement

df_gold['llama_8b_crowd_wisdom_agreement'] = llama_8b_crowd_wisdom_agreement

df_gold['llama_70b_crowd_wisdom_agreement'] = llama_70b_crowd_wisdom_agreement

In [35]:
df_gold

,Input,Answer,Confidence,All answers,Answer_3_classes,ChatGPT-4 5 classes,ChatGPT-4 3 classes,Model_agreement_with_human,model_agreement_with_human_3_classes,ChatGPT-3.5 5 classes,...,Model_agreement_with_human_3_classes_LLaMA_8B,Model_agreement_with_human_LLaMA_70B,Model_agreement_with_human_3_classes_LLaMA_70B,Model_agreement_with_human_3_classes_deberta_nli,human_human_agreement,human_crowd_wisdom_agreement,chatgpt4_crowd_wisdom_agreement,chatgpt3_5_crowd_wisdom_agreement,llama_8b_crowd_wisdom_agreement,llama_70b_crowd_wisdom_agreement
0,Text 1: Motivated and confident workforce is a...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,1.000000,Factual inconsistency,...,1.000000,0.800000,1.000000,1.000000,0.600000,0.800000,1.0,0.0,1.0,1.0
1,Text 1: We want broad support for agriculture ...,Surface contradiction,60.000,"['Consistent', 'Surface contradiction', 'Surfa...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,0.800000,Surface contradiction,...,0.800000,0.000000,0.800000,0.800000,0.300000,0.600000,0.0,1.0,0.0,0.0
2,Text 1: There should be a €29 ticket for every...,Surface contradiction,80.000,"['Indirect inconsistency', 'Surface contradict...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,1.000000,Factual inconsistency,...,1.000000,0.200000,1.000000,1.000000,0.600000,0.800000,0.0,0.0,0.0,0.0
3,Text 1: We support the implementation of pilot...,Unrelated,60.000,"['Indirect inconsistency', 'Unrelated', 'Factu...",Unrelated,Indirect inconsistency,Unrelated,0.200000,0.600000,Indirect inconsistency,...,0.600000,0.200000,0.600000,0.600000,0.300000,0.600000,0.0,0.0,1.0,0.0
4,Text 1: We fully support the conversion of add...,Indirect inconsistency,83.333,"['Unrelated', 'Indirect inconsistency', 'Indir...",Inconsistent,Indirect inconsistency,Inconsistent,0.833333,0.833333,Factual inconsistency,...,0.833333,0.833333,0.833333,0.833333,0.666667,0.833333,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,"Text 1: To bolster the economy, a major focus...",Indirect inconsistency,60.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Factual inconsistency,Inconsistent,0.400000,1.000000,Surface contradiction,...,1.000000,0.600000,1.000000,1.000000,0.400000,0.600000,0.0,0.0,0.0,1.0
694,Text 1: Students should have the free choice b...,Surface contradiction,60.000,"['Factual inconsistency', 'Factual inconsisten...",Inconsistent,Indirect inconsistency,Inconsistent,0.000000,1.000000,Surface contradiction,...,1.000000,0.000000,0.000000,1.000000,0.400000,0.000000,0.0,0.4,0.6,0.0
695,Text 1: Belonging to a certain social or ethni...,Indirect inconsistency,80.000,"['Indirect inconsistency', 'Indirect inconsist...",Inconsistent,Indirect inconsistency,Inconsistent,0.800000,0.800000,Surface contradiction,...,0.200000,0.200000,0.200000,0.200000,0.600000,0.800000,1.0,0.0,1.0,0.0
696,Text 1: The level of social basic security mus...,Factual inconsistency,40.000,"['Indirect inconsistency', 'Factual inconsiste...",Inconsistent,Indirect inconsistency,Inconsistent,0.200000,0.600000,Surface contradiction,...,0.600000,0.200000,0.600000,0.600000,0.200000,0.000000,0.0,0.0,0.0,0.0


In [36]:
# df_gold.to_csv(r"C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\qualtrics_survey\results\df_gold_all.csv", index=False)

In [15]:
df_gold.columns

Index(['Input', 'Answer', 'Confidence', 'All answers', 'Answer_3_classes',
       'ChatGPT-4 5 classes', 'ChatGPT-4 3 classes',
       'Model_agreement_with_human', 'model_agreement_with_human_3_classes',
       'ChatGPT-3.5 5 classes', 'ChatGPT-3.5 3 classes', 'LLaMA 5 classes',
       'LLaMA 3 classes', 'LLaMA 70B 5 classes', 'LLaMA 70B 3 classes',
       'deberta-v3-large-nli_3_classes',
       'Model_agreement_with_human_gpt3.5_turbo',
       'model_agreement_with_human_3_classes_gpt3.5_turbo',
       'Model_agreement_with_human_LLaMA_8B',
       'Model_agreement_with_human_3_classes_LLaMA_8B',
       'Model_agreement_with_human_LLaMA_70B',
       'Model_agreement_with_human_3_classes_LLaMA_70B',
       'Model_agreement_with_human_3_classes_deberta_nli',
       'human_human_agreement', 'human_crowd_wisdom_agreement',
       'chatgpt4_crowd_wisdom_agreement', 'chatgpt3_5_crowd_wisdom_agreement',
       'llama_8b_crowd_wisdom_agreement', 'llama_70b_crowd_wisdom_agreement',
       'ch

In [59]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\model_agreement_per_class.pkl', 'rb') as f:
    a = pickle.load(f)

In [60]:
a.keys()

dict_keys(['chatgpt_4', 'chatgpt_3.5_turbo', 'LLaMA 8B', 'LLaMA 70B'])

### Evaluate average agreement human-human per class + model-indiv human

In [54]:
avg_agreement_with_individuals = dict()
avg_agreement_with_individuals['5_classes'] = dict()

In [55]:
avg_agreement_with_individuals['5_classes']['humans'] = dict()
avg_agreement_with_individuals['5_classes']['humans']['total'] = np.mean(list(df_gold['human_human_agreement']))
for class_name in classes_5:
    avg_agreement_with_individuals['5_classes']['humans'][class_name] = np.mean(list(df_gold[df_gold['Answer']==class_name]['human_human_agreement']))

In [56]:
avg_agreement_with_individuals

{'5_classes': {'humans': {'total': 0.4564788283985992,
   'Unrelated': 0.5457627118644067,
   'Consistent': 0.6576388888888888,
   'Indirect inconsistency': 0.36964618249534453,
   'Factual inconsistency': 0.3450819672131148,
   'Surface contradiction': 0.4525804493017608}}}

In [61]:
model_to_column_name = [('chatgpt4', 'chatgpt4_individuals_agreement'),
                        ('chatgpt_3.5_turbo', 'chatgpt3_5_individuals_agreement'),
                        ('LLaMA 8B', 'llama_8b_individuals_agreement'),
                        ('LLaMA 70B', 'llama_70b_individuals_agreement')]

for model_name, column_name in model_to_column_name:
    avg_agreement_with_individuals['5_classes'][model_name] = dict()
    avg_agreement_with_individuals['5_classes'][model_name]['total'] = np.mean(list(df_gold[column_name]))
    for class_name in classes_5:
        avg_agreement_with_individuals['5_classes'][model_name][class_name] = np.mean(list(df_gold[df_gold['Answer']==class_name][column_name]))

In [62]:
avg_agreement_with_individuals

{'5_classes': {'humans': {'total': 0.4564788283985992,
   'Unrelated': 0.5457627118644067,
   'Consistent': 0.6576388888888888,
   'Indirect inconsistency': 0.36964618249534453,
   'Factual inconsistency': 0.3450819672131148,
   'Surface contradiction': 0.4525804493017608},
  'chatgpt4': {'total': 0.47101241642788916,
   'Unrelated': 0.5932203389830507,
   'Consistent': 0.7413194444444443,
   'Indirect inconsistency': 0.4927374301675978,
   'Factual inconsistency': 0.2653005464480875,
   'Surface contradiction': 0.3663023679417123},
  'chatgpt_3.5_turbo': {'total': 0.32507163323782234,
   'Unrelated': 0.13220338983050847,
   'Consistent': 0.32708333333333334,
   'Indirect inconsistency': 0.2325884543761639,
   'Factual inconsistency': 0.44890710382513654,
   'Surface contradiction': 0.4562841530054645},
  'LLaMA 8B': {'total': 0.33443170964660934,
   'Unrelated': 0.411864406779661,
   'Consistent': 0.6045138888888889,
   'Indirect inconsistency': 0.2122905027932961,
   'Factual inconsi

In [63]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\avg_agreement_with_individuals.pkl', 'wb') as f:
    pickle.dump(avg_agreement_with_individuals, f)

### Evaluate average agreement human-majority per class + model-majority per class

In [64]:
avg_agreement_with_majority = dict()
avg_agreement_with_majority['5_classes'] = dict()

In [65]:
avg_agreement_with_majority['5_classes']['humans'] = dict()
avg_agreement_with_majority['5_classes']['humans']['total'] = np.mean(list(df_gold['human_crowd_wisdom_agreement']))
for class_name in classes_5:
    avg_agreement_with_majority['5_classes']['humans'][class_name] = np.mean(list(df_gold[df_gold['Answer']==class_name]['human_crowd_wisdom_agreement']))

In [67]:
model_to_column_name = [('chatgpt4', 'chatgpt4_crowd_wisdom_agreement'),
                        ('chatgpt_3.5_turbo', 'chatgpt3_5_crowd_wisdom_agreement'),
                        ('LLaMA 8B', 'llama_8b_crowd_wisdom_agreement'),
                        ('LLaMA 70B', 'llama_70b_crowd_wisdom_agreement')]

for model_name, column_name in model_to_column_name:
    avg_agreement_with_majority['5_classes'][model_name] = dict()
    avg_agreement_with_majority['5_classes'][model_name]['total'] = np.mean(list(df_gold[column_name]))
    for class_name in classes_5:
        avg_agreement_with_majority['5_classes'][model_name][class_name] = np.mean(list(df_gold[df_gold['Answer']==class_name][column_name]))

In [68]:
avg_agreement_with_majority

{'5_classes': {'humans': {'total': 0.512989493791786,
   'Unrelated': 0.6101694915254237,
   'Consistent': 0.6763888888888889,
   'Indirect inconsistency': 0.4394785847299813,
   'Factual inconsistency': 0.3762295081967213,
   'Surface contradiction': 0.5276867030965391},
  'chatgpt4': {'total': 0.5196752626552054,
   'Unrelated': 0.664406779661017,
   'Consistent': 0.8229166666666666,
   'Indirect inconsistency': 0.6242085661080073,
   'Factual inconsistency': 0.2278688524590164,
   'Surface contradiction': 0.3595628415300547},
  'chatgpt_3.5_turbo': {'total': 0.3538681948424069,
   'Unrelated': 0.10169491525423731,
   'Consistent': 0.31875000000000003,
   'Indirect inconsistency': 0.23482309124767226,
   'Factual inconsistency': 0.5475409836065575,
   'Surface contradiction': 0.5222222222222223},
  'LLaMA 8B': {'total': 0.3530085959885387,
   'Unrelated': 0.4355932203389831,
   'Consistent': 0.6770833333333334,
   'Indirect inconsistency': 0.2011173184357542,
   'Factual inconsistenc

In [69]:
with open(r'C:\Users\Nursulu_1\Downloads\ContraDetect\automatic_stance_detection\data\model_evaluations\avg_agreement_with_majority.pkl', 'wb') as f:
    pickle.dump(avg_agreement_with_majority, f)